**Initial Configurations**

In [0]:
import json
import os

from pyspark.sql import SparkSession
from pyspark.ml import PipelineModel,Pipeline
from pyspark.sql import functions as F
from pyspark.sql.types import *

from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *
import sparknlp_jsl
import sparknlp

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")
pd.set_option("display.max_colwidth",100)

print('sparknlp.version : ',sparknlp.version())
print('sparknlp_jsl.version : ',sparknlp_jsl.version())

spark

sparknlp.version : 3.4.2
sparknlp_jsl.version : 3.4.2
Out[6]:

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.1.2 
 Master 
 spark://10.164.230.130:7077 
 AppName 
 Databricks Shell

In [0]:
spark._jvm.com.johnsnowlabs.util.start.registerListenerAndStartRefresh()

## Download Dataset

In [0]:
!wget -q https://raw.githubusercontent.com/iamvarol/blogposts/main/databricks/data/data.csv -P /dbfs/

In [0]:
%fs
ls

path,name,size
dbfs:/400.txt,400.txt,2669
dbfs:/400_rot.pdf,400_rot.pdf,2240141
dbfs:/ADE-NEG.txt,ADE-NEG.txt,2308469
dbfs:/ADE-NEG.txt.1,ADE-NEG.txt.1,2308469
dbfs:/AntBNC_lemmas_ver_001.txt,AntBNC_lemmas_ver_001.txt,1348552
dbfs:/AskAPatient.fold-0.test.txt,AskAPatient.fold-0.test.txt,33834
dbfs:/AskAPatient.fold-0.train.txt,AskAPatient.fold-0.train.txt,594794
dbfs:/AskAPatient.fold-0.validation.txt,AskAPatient.fold-0.validation.txt,32162
dbfs:/ClassifierDL_USE_20200727_e5/,ClassifierDL_USE_20200727_e5/,0
dbfs:/ClassifierDL_USE_20200923_e5/,ClassifierDL_USE_20200923_e5/,0


In [0]:
df = pd.read_csv('/dbfs/data.csv', sep=';')
df

Out[3]:

,subject_id,date,text,gender,dateOfBirth
0,19823,2167-02-25,Admission Date: [**2167-2-16**] Discharge Date: [**2167-2-24**]\n\nDate of Birth: [**2...,F,2099-05-05
1,19823,2167-11-27,Admission Date: [**2167-11-27**] Discharge Date: [**2167-12-9**]\n\nDate of Birth: [**...,F,2099-05-05
2,19823,2170-10-12,Admission Date: [**2170-9-19**] Discharge Date: [**2170-10-12**]\n\nDate of Birt...,F,2099-05-05
3,19823,2172-06-22,Admission Date: [**2172-6-13**] Discharge Date: [**2172-6-22**]\n\nDate of Birth...,F,2099-05-05
4,19823,2167-12-07,PATIENT/TEST INFORMATION:\nIndication: Aortic valve disease. Shortness of breath.\nHeight: (in) ...,F,2099-05-05
...,...,...,...,...,...
960,70004,2182-06-14,[**2182-6-14**] 10:45 AM\n MR HEAD W & W/O CONTRAST Clip ...,M,2127-12-06
961,70004,2182-06-25,FDG TUMOR IMAGING (PET-CT) Clip # [**Clip Number (Radiology...,M,2127-12-06
962,70004,2182-08-05,[**2182-8-5**] 11:46 AM\n MR HEAD W & W/O CONTRAST Clip #...,M,2127-12-06
963,70004,2182-08-23,FDG TUMOR IMAGING (PET-CT) Clip # [**Clip Number (Radiology...,M,2127-12-06


In [0]:
sparkDF=spark.createDataFrame(df) 
sparkDF.printSchema()
sparkDF.show()

root
-- subject_id: long (nullable = true)
-- date: string (nullable = true)
-- text: string (nullable = true)
-- gender: string (nullable = true)
-- dateOfBirth: string (nullable = true)

+----------+----------+--------------------+------+-----------+
subject_id| date| text|gender|dateOfBirth|
+----------+----------+--------------------+------+-----------+
 19823|2167-02-25|Admission Date: ...| F| 2099-05-05|
 19823|2167-11-27|Admission Date: ...| F| 2099-05-05|
 19823|2170-10-12|Admission Date: ...| F| 2099-05-05|
 19823|2172-06-22|Admission Date: ...| F| 2099-05-05|
 19823|2167-12-07|PATIENT/TEST INFO...| F| 2099-05-05|
 19823|2167-02-20|PATIENT/TEST INFO...| F| 2099-05-05|
 19823|2172-06-15|PATIENT/TEST INFO...| F| 2099-05-05|
 19823|2170-09-20|PATIENT/TEST INFO...| F| 2099-05-05|
 19823|2167-12-04|Atrial fibrillati...| F| 2099-05-05|
 19823|2167-12-03|Atrial fibrillati...| F| 2099-05-05|
 19823|2167-11-27|Atrial fibrillati...| F| 2099-05-05|
 19823|2167-02-25|Sinus rhythm with...| F| 2099-05-05|
 19823|2167-02-24|Sinus rhythm with...| F| 2099-05-05|
 19823|2167-02-23|Sinus rhythm. Lef...| F| 2099-05-05|
 19823|2167-02-22|Sinus rhythm with...| F| 2099-05-05|
 19823|2167-02-20|Sinus rhythm with...| F| 2099-05-05|
 19823|2167-02-17|Sinus rhythm. Lef...| F| 2099-05-05|
 19823|2172-06-22|Sinus rhythm
Bord...| F| 2099-05-05|
 19823|2172-06-21|Sinus tachycardia...| F| 2099-05-05|
 19823|2172-06-20|Sinus rhythm
Left...| F| 2099-05-05|
+----------+----------+--------------------+------+-----------+
only showing top 20 rows

## Posology RE Pipeline

In [0]:
documenter = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("documents")

sentencer = SentenceDetector()\
    .setInputCols(["documents"])\
    .setOutputCol("sentences")

tokenizer = sparknlp.annotators.Tokenizer()\
    .setInputCols(["sentences"])\
    .setOutputCol("tokens")

words_embedder = WordEmbeddingsModel()\
    .pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentences", "tokens"])\
    .setOutputCol("embeddings")

pos_tagger = PerceptronModel()\
    .pretrained("pos_clinical", "en", "clinical/models") \
    .setInputCols(["sentences", "tokens"])\
    .setOutputCol("pos_tags")

posology_ner = MedicalNerModel()\
    .pretrained("ner_posology", "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ners")   

posology_ner_converter = NerConverterInternal() \
    .setInputCols(["sentences", "tokens", "ners"]) \
    .setOutputCol("ner_chunks")

dependency_parser = DependencyParserModel()\
    .pretrained("dependency_conllu", "en")\
    .setInputCols(["sentences", "pos_tags", "tokens"])\
    .setOutputCol("dependencies")

reModel = RelationExtractionModel()\
    .pretrained("posology_re")\
    .setInputCols(["embeddings", "pos_tags", "ner_chunks", "dependencies"])\
    .setOutputCol("posology_relations")\
    .setMaxSyntacticDistance(4)

pipeline = Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    posology_ner,
    posology_ner_converter,
    dependency_parser,
    reModel
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = pipeline.fit(empty_data)

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][OK!]
pos_clinical download started this may take some time.
Approximate size to download 1.5 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][OK!]
ner_posology download started this may take some time.
Approximate size to download 13.8 MB
[ | ][ / ][ — ][ \ ][OK!]
dependency_conllu download started this may take some time.
Approximate size to download 16.7 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][OK!]

In [0]:
results = model.transform(sparkDF)
results.printSchema()

root
-- subject_id: long (nullable = true)
-- date: string (nullable = true)
-- text: string (nullable = true)
-- gender: string (nullable = true)
-- dateOfBirth: string (nullable = true)
-- documents: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- sentences: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- tokens: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- embeddings: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- pos_tags: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- ners: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- ner_chunks: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- dependencies: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- posology_relations: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: 

In [0]:
results.select('posology_relations.metadata').show(5)

+--------------------+
 metadata|
+--------------------+
[{chunk2 -> nebul...|
[{chunk2 -> face ...|
[{chunk2 -> PRBC,...|
[{chunk2 -> for 1...|
 []|
+--------------------+
only showing top 5 rows

In [0]:
results.select('subject_id','date', F.explode(F.arrays_zip('posology_relations.result', 'posology_relations.metadata')).alias("cols")).show()

+----------+----------+--------------------+
subject_id| date| cols|
+----------+----------+--------------------+
 19823|2167-02-25|{DRUG-FORM, {chun...|
 19823|2167-02-25|{DRUG-FORM, {chun...|
 19823|2167-02-25|{STRENGTH-DRUG, {...|
 19823|2167-02-25|{ROUTE-DRUG, {chu...|
 19823|2167-02-25|{DRUG-STRENGTH, {...|
 19823|2167-02-25|{DRUG-ROUTE, {chu...|
 19823|2167-02-25|{DRUG-FREQUENCY, ...|
 19823|2167-02-25|{DRUG-STRENGTH, {...|
 19823|2167-02-25|{DRUG-FREQUENCY, ...|
 19823|2167-02-25|{STRENGTH-DRUG, {...|
 19823|2167-02-25|{ROUTE-DRUG, {chu...|
 19823|2167-02-25|{FREQUENCY-DRUG, ...|
 19823|2167-02-25|{FREQUENCY-DRUG, ...|
 19823|2167-02-25|{FREQUENCY-DRUG, ...|
 19823|2167-02-25|{FREQUENCY-DRUG, ...|
 19823|2167-02-25|{FREQUENCY-DRUG, ...|
 19823|2167-02-25|{DRUG-STRENGTH, {...|
 19823|2167-02-25|{DRUG-ROUTE, {chu...|
 19823|2167-02-25|{DRUG-FREQUENCY, ...|
 19823|2167-02-25|{DRUG-STRENGTH, {...|
+----------+----------+--------------------+
only showing top 20 rows

In [0]:
result_df = results.select('subject_id','date',F.explode(F.arrays_zip(results.posology_relations.result, results.posology_relations.metadata)).alias("cols")) \
                   .select('subject_id','date',F.expr("cols['0']").alias("relation"),
                                               F.expr("cols['1']['entity1']").alias("entity1"),
                                               F.expr("cols['1']['entity1_begin']").alias("entity1_begin"),
                                               F.expr("cols['1']['entity1_end']").alias("entity1_end"),
                                               F.expr("cols['1']['chunk1']").alias("chunk1"),
                                               F.expr("cols['1']['entity2']").alias("entity2"),
                                               F.expr("cols['1']['entity2_begin']").alias("entity2_begin"),
                                               F.expr("cols['1']['entity2_end']").alias("entity2_end"),
                                               F.expr("cols['1']['chunk2']").alias("chunk2"),
                                               F.expr("cols['1']['confidence']").alias("confidence"))

In [0]:
result_df.show()

+----------+----------+--------------+---------+-------------+-----------+----------+---------+-------------+-----------+----------+----------+
subject_id| date| relation| entity1|entity1_begin|entity1_end| chunk1| entity2|entity2_begin|entity2_end| chunk2|confidence|
+----------+----------+--------------+---------+-------------+-----------+----------+---------+-------------+-----------+----------+----------+
 19823|2167-02-25| DRUG-FORM| DRUG| 1391| 1399| Albuterol| FORM| 1414| 1423|nebulizers| 1.0|
 19823|2167-02-25| DRUG-FORM| DRUG| 1405| 1412| Atrovent| FORM| 1414| 1423|nebulizers| 1.0|
 19823|2167-02-25| STRENGTH-DRUG| STRENGTH| 1539| 1543| 40 mg| DRUG| 1551| 1555| Lasix| 1.0|
 19823|2167-02-25| ROUTE-DRUG| ROUTE| 1548| 1549| IV| DRUG| 1551| 1555| Lasix| 1.0|
 19823|2167-02-25| DRUG-STRENGTH| DRUG| 2336| 2341| Amaryl| STRENGTH| 2343| 2348| 2.0 mg| 1.0|
 19823|2167-02-25| DRUG-ROUTE| DRUG| 2336| 2341| Amaryl| ROUTE| 2350| 2351| po| 1.0|
 19823|2167-02-25|DRUG-FREQUENCY| DRUG| 2336| 2341| Amaryl|FREQUENCY| 2353| 2355| bid| 1.0|
 19823|2167-02-25| DRUG-STRENGTH| DRUG| 2336| 2341| Amaryl| STRENGTH| 2372| 2379| 1,000 mg| 1.0|
 19823|2167-02-25|DRUG-FREQUENCY| DRUG| 2336| 2341| Amaryl|FREQUENCY| 2384| 2386| bid| 1.0|
 19823|2167-02-25| STRENGTH-DRUG| STRENGTH| 2343| 2348| 2.0 mg| DRUG| 2361| 2370|Glucophage| 1.0|
 19823|2167-02-25| ROUTE-DRUG| ROUTE| 2350| 2351| po| DRUG| 2361| 2370|Glucophage| 1.0|
 19823|2167-02-25|FREQUENCY-DRUG|FREQUENCY| 2353| 2355| bid| DRUG| 2361| 2370|Glucophage| 1.0|
 19823|2167-02-25|FREQUENCY-DRUG|FREQUENCY| 2353| 2355| bid| DRUG| 2392| 2400| Singulair| 1.0|
 19823|2167-02-25|FREQUENCY-DRUG|FREQUENCY| 2353| 2355| bid| DRUG| 2421| 2425| Zocor| 1.0|
 19823|2167-02-25|FREQUENCY-DRUG|FREQUENCY| 2353| 2355| bid| DRUG| 2446| 2451| Hyzaar| 1.0|
 19823|2167-02-25|FREQUENCY-DRUG|FREQUENCY| 2353| 2355| bid| DRUG| 2471| 2478| Atenolol| 1.0|
 19823|2167-02-25| DRUG-STRENGTH| DRUG| 2361| 2370|Glucophage| STRENGTH| 2372| 2379| 1,000 mg| 1.0|
 19823|2167-02-25| DRUG-ROUTE| DRUG| 2361| 2370|Glucophage| ROUTE| 2381| 2382| po| 1.0|
 19823|2167-02-25|DRUG-FREQUENCY| DRUG| 2361| 2370|Glucophage|FREQUENCY| 2384| 2386| bid| 1.0|
 19823|2167-02-25| DRUG-STRENGTH| DRUG| 2361| 2370|Glucophage| STRENGTH| 2402| 2406| 10 mg| 1.0|
+----------+----------+--------------+---------+-------------+-----------+----------+---------+-------------+-----------+----------+----------+
only showing top 20 rows

In [0]:
pd_result = result_df.toPandas()
pd_result

Out[9]:

,subject_id,date,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
0,19823,2167-02-25,DRUG-FORM,DRUG,1391,1399,Albuterol,FORM,1414,1423,nebulizers,1.0
1,19823,2167-02-25,DRUG-FORM,DRUG,1405,1412,Atrovent,FORM,1414,1423,nebulizers,1.0
2,19823,2167-02-25,STRENGTH-DRUG,STRENGTH,1539,1543,40 mg,DRUG,1551,1555,Lasix,1.0
3,19823,2167-02-25,ROUTE-DRUG,ROUTE,1548,1549,IV,DRUG,1551,1555,Lasix,1.0
4,19823,2167-02-25,DRUG-STRENGTH,DRUG,2336,2341,Amaryl,STRENGTH,2343,2348,2.0 mg,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2777,70004,2182-08-05,ROUTE-DRUG,ROUTE,545,546,IV,DRUG,548,555,contrast,1.0
2778,70004,2182-08-05,DOSAGE-DRUG,DOSAGE,942,946,20 cc,DRUG,951,959,Magnevist,1.0
2779,70004,2182-08-05,DRUG-ROUTE,DRUG,951,959,Magnevist,ROUTE,961,971,intravenous,1.0
2780,70004,2182-08-16,ROUTE-DRUG,ROUTE,475,476,IV,DRUG,478,485,CONTRAST,1.0


In [0]:
dbutils.fs.mkdirs("/dbfs/dbr_demo")

Out[12]: True

In [0]:
dbutils.fs.ls("/dbfs/")

Out[18]: [FileInfo(path='dbfs:/dbfs/FileStore/', name='FileStore/', size=0),
 FileInfo(path='dbfs:/dbfs/FileSystem/', name='FileSystem/', size=0),
 FileInfo(path='dbfs:/dbfs/dbr_demo/', name='dbr_demo/', size=0),
 FileInfo(path='dbfs:/dbfs/home/', name='home/', size=0),
 FileInfo(path='dbfs:/dbfs/models/', name='models/', size=0),
 FileInfo(path='dbfs:/dbfs/nlu_models/', name='nlu_models/', size=0),
 FileInfo(path='dbfs:/dbfs/tmp/', name='tmp/', size=0)]

In [0]:
dbutils.fs.ls("/dbfs/FileStore")

Out[21]: [FileInfo(path='dbfs:/dbfs/FileStore/log/', name='log/', size=0),
 FileInfo(path='dbfs:/dbfs/FileStore/logs/', name='logs/', size=0)]

In [0]:
outname = 'posology_re_results.csv'
outdir = '/dbfs/FileStore/'
pd_result.to_csv(outdir+outname, index=False, encoding="utf-8")

In [0]:
temp = pd.read_csv('/dbfs/FileStore/posology_re_results.csv')
temp

Out[22]:

,subject_id,date,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
0,19823,2167-02-25,DRUG-FORM,DRUG,1391,1399,Albuterol,FORM,1414,1423,nebulizers,1.0
1,19823,2167-02-25,DRUG-FORM,DRUG,1405,1412,Atrovent,FORM,1414,1423,nebulizers,1.0
2,19823,2167-02-25,STRENGTH-DRUG,STRENGTH,1539,1543,40 mg,DRUG,1551,1555,Lasix,1.0
3,19823,2167-02-25,ROUTE-DRUG,ROUTE,1548,1549,IV,DRUG,1551,1555,Lasix,1.0
4,19823,2167-02-25,DRUG-STRENGTH,DRUG,2336,2341,Amaryl,STRENGTH,2343,2348,2.0 mg,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2777,70004,2182-08-05,ROUTE-DRUG,ROUTE,545,546,IV,DRUG,548,555,contrast,1.0
2778,70004,2182-08-05,DOSAGE-DRUG,DOSAGE,942,946,20 cc,DRUG,951,959,Magnevist,1.0
2779,70004,2182-08-05,DRUG-ROUTE,DRUG,951,959,Magnevist,ROUTE,961,971,intravenous,1.0
2780,70004,2182-08-16,ROUTE-DRUG,ROUTE,475,476,IV,DRUG,478,485,CONTRAST,1.0


## RxNorm Code Extraction From Re_Results

In [0]:
import pandas as pd
pd_RE = pd.read_csv('/dbfs/FileStore/posology_re_results.csv')
pd_RE

Out[2]:

,subject_id,date,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
0,19823,2167-02-25,DRUG-FORM,DRUG,1391,1399,Albuterol,FORM,1414,1423,nebulizers,1.0
1,19823,2167-02-25,DRUG-FORM,DRUG,1405,1412,Atrovent,FORM,1414,1423,nebulizers,1.0
2,19823,2167-02-25,STRENGTH-DRUG,STRENGTH,1539,1543,40 mg,DRUG,1551,1555,Lasix,1.0
3,19823,2167-02-25,ROUTE-DRUG,ROUTE,1548,1549,IV,DRUG,1551,1555,Lasix,1.0
4,19823,2167-02-25,DRUG-STRENGTH,DRUG,2336,2341,Amaryl,STRENGTH,2343,2348,2.0 mg,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2777,70004,2182-08-05,ROUTE-DRUG,ROUTE,545,546,IV,DRUG,548,555,contrast,1.0
2778,70004,2182-08-05,DOSAGE-DRUG,DOSAGE,942,946,20 cc,DRUG,951,959,Magnevist,1.0
2779,70004,2182-08-05,DRUG-ROUTE,DRUG,951,959,Magnevist,ROUTE,961,971,intravenous,1.0
2780,70004,2182-08-16,ROUTE-DRUG,ROUTE,475,476,IV,DRUG,478,485,CONTRAST,1.0


In [0]:
sp_RE = spark.createDataFrame(pd_RE)
sp_RE.show(20)

+----------+----------+--------------+---------+-------------+-----------+----------+---------+-------------+-----------+----------+----------+
subject_id| date| relation| entity1|entity1_begin|entity1_end| chunk1| entity2|entity2_begin|entity2_end| chunk2|confidence|
+----------+----------+--------------+---------+-------------+-----------+----------+---------+-------------+-----------+----------+----------+
 19823|2167-02-25| DRUG-FORM| DRUG| 1391| 1399| Albuterol| FORM| 1414| 1423|nebulizers| 1.0|
 19823|2167-02-25| DRUG-FORM| DRUG| 1405| 1412| Atrovent| FORM| 1414| 1423|nebulizers| 1.0|
 19823|2167-02-25| STRENGTH-DRUG| STRENGTH| 1539| 1543| 40 mg| DRUG| 1551| 1555| Lasix| 1.0|
 19823|2167-02-25| ROUTE-DRUG| ROUTE| 1548| 1549| IV| DRUG| 1551| 1555| Lasix| 1.0|
 19823|2167-02-25| DRUG-STRENGTH| DRUG| 2336| 2341| Amaryl| STRENGTH| 2343| 2348| 2.0 mg| 1.0|
 19823|2167-02-25| DRUG-ROUTE| DRUG| 2336| 2341| Amaryl| ROUTE| 2350| 2351| po| 1.0|
 19823|2167-02-25|DRUG-FREQUENCY| DRUG| 2336| 2341| Amaryl|FREQUENCY| 2353| 2355| bid| 1.0|
 19823|2167-02-25| DRUG-STRENGTH| DRUG| 2336| 2341| Amaryl| STRENGTH| 2372| 2379| 1,000 mg| 1.0|
 19823|2167-02-25|DRUG-FREQUENCY| DRUG| 2336| 2341| Amaryl|FREQUENCY| 2384| 2386| bid| 1.0|
 19823|2167-02-25| STRENGTH-DRUG| STRENGTH| 2343| 2348| 2.0 mg| DRUG| 2361| 2370|Glucophage| 1.0|
 19823|2167-02-25| ROUTE-DRUG| ROUTE| 2350| 2351| po| DRUG| 2361| 2370|Glucophage| 1.0|
 19823|2167-02-25|FREQUENCY-DRUG|FREQUENCY| 2353| 2355| bid| DRUG| 2361| 2370|Glucophage| 1.0|
 19823|2167-02-25|FREQUENCY-DRUG|FREQUENCY| 2353| 2355| bid| DRUG| 2392| 2400| Singulair| 1.0|
 19823|2167-02-25|FREQUENCY-DRUG|FREQUENCY| 2353| 2355| bid| DRUG| 2421| 2425| Zocor| 1.0|
 19823|2167-02-25|FREQUENCY-DRUG|FREQUENCY| 2353| 2355| bid| DRUG| 2446| 2451| Hyzaar| 1.0|
 19823|2167-02-25|FREQUENCY-DRUG|FREQUENCY| 2353| 2355| bid| DRUG| 2471| 2478| Atenolol| 1.0|
 19823|2167-02-25| DRUG-STRENGTH| DRUG| 2361| 2370|Glucophage| STRENGTH| 2372| 2379| 1,000 mg| 1.0|
 19823|2167-02-25| DRUG-ROUTE| DRUG| 2361| 2370|Glucophage| ROUTE| 2381| 2382| po| 1.0|
 19823|2167-02-25|DRUG-FREQUENCY| DRUG| 2361| 2370|Glucophage|FREQUENCY| 2384| 2386| bid| 1.0|
 19823|2167-02-25| DRUG-STRENGTH| DRUG| 2361| 2370|Glucophage| STRENGTH| 2402| 2406| 10 mg| 1.0|
+----------+----------+--------------+---------+-------------+-----------+----------+---------+-------------+-----------+----------+----------+
only showing top 20 rows

In [0]:
sp_RE.rdd.getNumPartitions()

Out[4]: 32

In [0]:
# drug + strength or form
from pyspark.sql.functions import when, col

sp_RE_results = sp_RE.withColumn('rx_text',
  when( (F.col('entity1')=='DRUG') & ((F.col('entity2')=='FORM') | (F.col('entity2')=='STRENGTH') | (F.col('entity2')=='DOSAGE') ), F.concat(F.col('chunk1'),F.lit(' '), F.col('chunk2')))
 .when( ((F.col('entity1')=='FORM') | (F.col('entity1')=='STRENGTH') | (F.col('entity1')=='DOSAGE') ) & (F.col('entity2')=='DRUG'), F.concat(F.col('chunk2'),F.lit(' '), F.col('chunk1')))
 .when( (F.col('entity1')=='DRUG') & ((F.col('entity2')!='FORM') & (F.col('entity2')!='STRENGTH') & (F.col('entity2')!='DOSAGE') ), F.col('chunk1'))
 .when( (F.col('entity2')=='DRUG') & ((F.col('entity1')!='FORM') & (F.col('entity1')!='STRENGTH') & (F.col('entity1')!='DOSAGE') ), F.col('chunk2'))
                   .otherwise(F.lit(' '))
                   )

sp_RE_results.show(20,70)

+----------+----------+--------------+---------+-------------+-----------+----------+---------+-------------+-----------+----------+----------+--------------------+
subject_id| date| relation| entity1|entity1_begin|entity1_end| chunk1| entity2|entity2_begin|entity2_end| chunk2|confidence| rx_text|
+----------+----------+--------------+---------+-------------+-----------+----------+---------+-------------+-----------+----------+----------+--------------------+
 19823|2167-02-25| DRUG-FORM| DRUG| 1391| 1399| Albuterol| FORM| 1414| 1423|nebulizers| 1.0|Albuterol nebulizers|
 19823|2167-02-25| DRUG-FORM| DRUG| 1405| 1412| Atrovent| FORM| 1414| 1423|nebulizers| 1.0| Atrovent nebulizers|
 19823|2167-02-25| STRENGTH-DRUG| STRENGTH| 1539| 1543| 40 mg| DRUG| 1551| 1555| Lasix| 1.0| Lasix 40 mg|
 19823|2167-02-25| ROUTE-DRUG| ROUTE| 1548| 1549| IV| DRUG| 1551| 1555| Lasix| 1.0| Lasix|
 19823|2167-02-25| DRUG-STRENGTH| DRUG| 2336| 2341| Amaryl| STRENGTH| 2343| 2348| 2.0 mg| 1.0| Amaryl 2.0 mg|
 19823|2167-02-25| DRUG-ROUTE| DRUG| 2336| 2341| Amaryl| ROUTE| 2350| 2351| po| 1.0| Amaryl|
 19823|2167-02-25|DRUG-FREQUENCY| DRUG| 2336| 2341| Amaryl|FREQUENCY| 2353| 2355| bid| 1.0| Amaryl|
 19823|2167-02-25| DRUG-STRENGTH| DRUG| 2336| 2341| Amaryl| STRENGTH| 2372| 2379| 1,000 mg| 1.0| Amaryl 1,000 mg|
 19823|2167-02-25|DRUG-FREQUENCY| DRUG| 2336| 2341| Amaryl|FREQUENCY| 2384| 2386| bid| 1.0| Amaryl|
 19823|2167-02-25| STRENGTH-DRUG| STRENGTH| 2343| 2348| 2.0 mg| DRUG| 2361| 2370|Glucophage| 1.0| Glucophage 2.0 mg|
 19823|2167-02-25| ROUTE-DRUG| ROUTE| 2350| 2351| po| DRUG| 2361| 2370|Glucophage| 1.0| Glucophage|
 19823|2167-02-25|FREQUENCY-DRUG|FREQUENCY| 2353| 2355| bid| DRUG| 2361| 2370|Glucophage| 1.0| Glucophage|
 19823|2167-02-25|FREQUENCY-DRUG|FREQUENCY| 2353| 2355| bid| DRUG| 2392| 2400| Singulair| 1.0| Singulair|
 19823|2167-02-25|FREQUENCY-DRUG|FREQUENCY| 2353| 2355| bid| DRUG| 2421| 2425| Zocor| 1.0| Zocor|
 19823|2167-02-25|FREQUENCY-DRUG|FREQUENCY| 2353| 2355| bid| DRUG| 2446| 2451| Hyzaar| 1.0| Hyzaar|
 19823|2167-02-25|FREQUENCY-DRUG|FREQUENCY| 2353| 2355| bid| DRUG| 2471| 2478| Atenolol| 1.0| Atenolol|
 19823|2167-02-25| DRUG-STRENGTH| DRUG| 2361| 2370|Glucophage| STRENGTH| 2372| 2379| 1,000 mg| 1.0| Glucophage 1,000 mg|
 19823|2167-02-25| DRUG-ROUTE| DRUG| 2361| 2370|Glucophage| ROUTE| 2381| 2382| po| 1.0| Glucophage|
 19823|2167-02-25|DRUG-FREQUENCY| DRUG| 2361| 2370|Glucophage|FREQUENCY| 2384| 2386| bid| 1.0| Glucophage|
 19823|2167-02-25| DRUG-STRENGTH| DRUG| 2361| 2370|Glucophage| STRENGTH| 2402| 2406| 10 mg| 1.0| Glucophage 10 mg|
+----------+----------+--------------+---------+-------------+-----------+----------+---------+-------------+-----------+----------+----------+--------------------+
only showing top 20 rows

In [0]:
documentAssembler = DocumentAssembler()\
      .setInputCol("rx_text")\
      .setOutputCol("ner_chunk")

sbert_embedder = BertSentenceEmbeddings.pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("sentence_embeddings")
    
rxnorm_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_rxnorm_augmented","en", "clinical/models") \
      .setInputCols(["ner_chunk", "sentence_embeddings"]) \
      .setOutputCol("rxnorm_code")\
      .setDistanceFunction("EUCLIDEAN")

rxnorm_pipelineModel = PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        rxnorm_resolver])

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][OK!]
sbiobertresolve_rxnorm_augmented download started this may take some time.
Approximate size to download 930.9 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][OK!]

In [0]:
rxnorm_results = rxnorm_pipelineModel.transform(sp_RE_results)
rxnorm_results.printSchema(), rxnorm_results.rdd.getNumPartitions()

root
-- subject_id: long (nullable = true)
-- date: string (nullable = true)
-- relation: string (nullable = true)
-- entity1: string (nullable = true)
-- entity1_begin: long (nullable = true)
-- entity1_end: long (nullable = true)
-- chunk1: string (nullable = true)
-- entity2: string (nullable = true)
-- entity2_begin: long (nullable = true)
-- entity2_end: long (nullable = true)
-- chunk2: string (nullable = true)
-- confidence: double (nullable = true)
-- rx_text: string (nullable = true)
-- ner_chunk: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- sentence_embeddings: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- rxnorm_code: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)

Out[14]: (None, 32)

In [0]:
sp_rxnorm_result = rxnorm_results.select('subject_id','date', 'relation', 'entity1', 'entity1_begin','entity1_end',  'chunk1', 'entity2', 'entity2_begin', 'entity2_end', 
                                         'chunk2', 'confidence', 'rx_text', 
                                         F.explode(F.arrays_zip(rxnorm_results.ner_chunk.result, 
                                                                rxnorm_results.ner_chunk.metadata, 
                                                                rxnorm_results.rxnorm_code.result, 
                                                                rxnorm_results.rxnorm_code.metadata)).alias("cols")) \
                                     .select('subject_id','date', 'relation', 'entity1', 'entity1_begin','entity1_end',  'chunk1', 'entity2', 'entity2_begin', 'entity2_end',
                                             'chunk2', 'confidence', 'rx_text',
                                             F.expr("cols['1']['sentence']").alias("sent_id"),
                                             F.expr("cols['0']").alias("ner_chunk"),
                                             F.expr("cols['1']['entity']").alias("entity"), 
                                             F.expr("cols['2']").alias('rxnorm_code'),
                                             F.expr("cols['3']['all_k_results']").alias("all_codes"),
                                             F.expr("cols['3']['all_k_resolutions']").alias("resolutions"))

In [0]:
sp_rxnorm_result.show()

+----------+----------+--------------+---------+-------------+-----------+----------+---------+-------------+-----------+----------+----------+--------------------+-------+--------------------+------+-----------+--------------------+--------------------+
subject_id| date| relation| entity1|entity1_begin|entity1_end| chunk1| entity2|entity2_begin|entity2_end| chunk2|confidence| rx_text|sent_id| ner_chunk|entity|rxnorm_code| all_codes| resolutions|
+----------+----------+--------------+---------+-------------+-----------+----------+---------+-------------+-----------+----------+----------+--------------------+-------+--------------------+------+-----------+--------------------+--------------------+
 19823|2167-02-25| DRUG-FORM| DRUG| 1391| 1399| Albuterol| FORM| 1414| 1423|nebulizers| 1.0|Albuterol nebulizers| 0|Albuterol nebulizers| null| 2108226|2108226:::1154602...|albuterol Inhalat...|
 19823|2167-02-25| DRUG-FORM| DRUG| 1405| 1412| Atrovent| FORM| 1414| 1423|nebulizers| 1.0| Atrovent nebulizers| 0| Atrovent nebulizers| null| 2108451|2108451:::1173573...|ipratropium Inhal...|
 19823|2167-02-25| STRENGTH-DRUG| STRENGTH| 1539| 1543| 40 mg| DRUG| 1551| 1555| Lasix| 1.0| Lasix 40 mg| 0| Lasix 40 mg| null| 200809|200809:::617319::...|furosemide 40 MG ...|
 19823|2167-02-25| ROUTE-DRUG| ROUTE| 1548| 1549| IV| DRUG| 1551| 1555| Lasix| 1.0| Lasix| 0| Lasix| null| 202991|202991:::151963::...|Lasix:::Lasma:::l...|
 19823|2167-02-25| DRUG-STRENGTH| DRUG| 2336| 2341| Amaryl| STRENGTH| 2343| 2348| 2.0 mg| 1.0| Amaryl 2.0 mg| 0| Amaryl 2.0 mg| null| 901295|901295:::153591::...|sodium fluoride 2...|
 19823|2167-02-25| DRUG-ROUTE| DRUG| 2336| 2341| Amaryl| ROUTE| 2350| 2351| po| 1.0| Amaryl| 0| Amaryl| null| 215221|215221:::135820::...|Amilac:::Aventyl:...|
 19823|2167-02-25|DRUG-FREQUENCY| DRUG| 2336| 2341| Amaryl|FREQUENCY| 2353| 2355| bid| 1.0| Amaryl| 0| Amaryl| null| 215221|215221:::135820::...|Amilac:::Aventyl:...|
 19823|2167-02-25| DRUG-STRENGTH| DRUG| 2336| 2341| Amaryl| STRENGTH| 2372| 2379| 1,000 mg| 1.0| Amaryl 1,000 mg| 0| Amaryl 1,000 mg| null| 106248|106248:::1549223:...|hydrocortisone 1 ...|
 19823|2167-02-25|DRUG-FREQUENCY| DRUG| 2336| 2341| Amaryl|FREQUENCY| 2384| 2386| bid| 1.0| Amaryl| 0| Amaryl| null| 215221|215221:::135820::...|Amilac:::Aventyl:...|
 19823|2167-02-25| STRENGTH-DRUG| STRENGTH| 2343| 2348| 2.0 mg| DRUG| 2361| 2370|Glucophage| 1.0| Glucophage 2.0 mg| 0| Glucophage 2.0 mg| null| 865570|865570:::201058::...|glipizide 2.5 MG ...|
 19823|2167-02-25| ROUTE-DRUG| ROUTE| 2350| 2351| po| DRUG| 2361| 2370|Glucophage| 1.0| Glucophage| 0| Glucophage| null| 151827|151827:::1423808:...|Glucophage:::Glyc...|
 19823|2167-02-25|FREQUENCY-DRUG|FREQUENCY| 2353| 2355| bid| DRUG| 2361| 2370|Glucophage| 1.0| Glucophage| 0| Glucophage| null| 151827|151827:::1423808:...|Glucophage:::Glyc...|
 19823|2167-02-25|FREQUENCY-DRUG|FREQUENCY| 2353| 2355| bid| DRUG| 2392| 2400| Singulair| 1.0| Singulair| 0| Singulair| null| 152894|152894:::417282::...|Serdolect:::neo-h...|
 19823|2167-02-25|FREQUENCY-DRUG|FREQUENCY| 2353| 2355| bid| DRUG| 2421| 2425| Zocor| 1.0| Zocor| 0| Zocor| null| 196503|196503:::858343::...|Zocor:::Zipsor:::...|
 19823|2167-02-25|FREQUENCY-DRUG|FREQUENCY| 2353| 2355| bid| DRUG| 2446| 2451| Hyzaar| 1.0| Hyzaar| 0| Hyzaar| null| 217681|217681:::1043619:...|Hyzaar:::eucalypt...|
 19823|2167-02-25|FREQUENCY-DRUG|FREQUENCY| 2353| 2355| bid| DRUG| 2471| 2478| Atenolol| 1.0| Atenolol| 0| Atenolol| null| 218896|218896:::370619::...|OptiPranolol:::at...|
 19823|2167-02-25| DRUG-STRENGTH| DRUG| 2361| 2370|Glucophage| STRENGTH| 2372| 2379| 1,000 mg| 1.0| Glucophage 1,000 mg| 0| Glucophage 1,000 mg| null| 153095|153095:::1721278:...|glucagon (rDNA) 1...|
 19823|2167-02-25| DRUG-ROUTE| DRUG| 2361| 2370|Glucophage| ROUTE| 2381| 2382| po| 1.0| Glucophage| 0| Glucophage| null| 151827|151827:::1423808:...|Glucophage:::Glyc...|
 19823|2167-02-25|DRUG-FREQUENCY| DRUG| 2361| 2370|Glucophage|FREQUENCY| 2384| 2386| bid| 1.0| Glucophage|

In [0]:
sp_rxnorm_result = sp_rxnorm_result.withColumn('all_codes', F.split(F.col('all_codes'), ':::'))\
                                    .withColumn('resolutions', F.split(F.col('resolutions'), ':::'))\

sp_rxnorm_result.show()

+----------+----------+--------------+---------+-------------+-----------+----------+---------+-------------+-----------+----------+----------+--------------------+-------+--------------------+------+-----------+--------------------+--------------------+
subject_id| date| relation| entity1|entity1_begin|entity1_end| chunk1| entity2|entity2_begin|entity2_end| chunk2|confidence| rx_text|sent_id| ner_chunk|entity|rxnorm_code| all_codes| resolutions|
+----------+----------+--------------+---------+-------------+-----------+----------+---------+-------------+-----------+----------+----------+--------------------+-------+--------------------+------+-----------+--------------------+--------------------+
 19823|2167-02-25| DRUG-FORM| DRUG| 1391| 1399| Albuterol| FORM| 1414| 1423|nebulizers| 1.0|Albuterol nebulizers| 0|Albuterol nebulizers| null| 2108226|[2108226, 1154602...|[albuterol Inhala...|
 19823|2167-02-25| DRUG-FORM| DRUG| 1405| 1412| Atrovent| FORM| 1414| 1423|nebulizers| 1.0| Atrovent nebulizers| 0| Atrovent nebulizers| null| 2108451|[2108451, 1173573...|[ipratropium Inha...|
 19823|2167-02-25| STRENGTH-DRUG| STRENGTH| 1539| 1543| 40 mg| DRUG| 1551| 1555| Lasix| 1.0| Lasix 40 mg| 0| Lasix 40 mg| null| 200809|[200809, 617319, ...|[furosemide 40 MG...|
 19823|2167-02-25| ROUTE-DRUG| ROUTE| 1548| 1549| IV| DRUG| 1551| 1555| Lasix| 1.0| Lasix| 0| Lasix| null| 202991|[202991, 151963, ...|[Lasix, Lasma, la...|
 19823|2167-02-25| DRUG-STRENGTH| DRUG| 2336| 2341| Amaryl| STRENGTH| 2343| 2348| 2.0 mg| 1.0| Amaryl 2.0 mg| 0| Amaryl 2.0 mg| null| 901295|[901295, 153591, ...|[sodium fluoride ...|
 19823|2167-02-25| DRUG-ROUTE| DRUG| 2336| 2341| Amaryl| ROUTE| 2350| 2351| po| 1.0| Amaryl| 0| Amaryl| null| 215221|[215221, 135820, ...|[Amilac, Aventyl,...|
 19823|2167-02-25|DRUG-FREQUENCY| DRUG| 2336| 2341| Amaryl|FREQUENCY| 2353| 2355| bid| 1.0| Amaryl| 0| Amaryl| null| 215221|[215221, 135820, ...|[Amilac, Aventyl,...|
 19823|2167-02-25| DRUG-STRENGTH| DRUG| 2336| 2341| Amaryl| STRENGTH| 2372| 2379| 1,000 mg| 1.0| Amaryl 1,000 mg| 0| Amaryl 1,000 mg| null| 106248|[106248, 1549223,...|[hydrocortisone 1...|
 19823|2167-02-25|DRUG-FREQUENCY| DRUG| 2336| 2341| Amaryl|FREQUENCY| 2384| 2386| bid| 1.0| Amaryl| 0| Amaryl| null| 215221|[215221, 135820, ...|[Amilac, Aventyl,...|
 19823|2167-02-25| STRENGTH-DRUG| STRENGTH| 2343| 2348| 2.0 mg| DRUG| 2361| 2370|Glucophage| 1.0| Glucophage 2.0 mg| 0| Glucophage 2.0 mg| null| 865570|[865570, 201058, ...|[glipizide 2.5 MG...|
 19823|2167-02-25| ROUTE-DRUG| ROUTE| 2350| 2351| po| DRUG| 2361| 2370|Glucophage| 1.0| Glucophage| 0| Glucophage| null| 151827|[151827, 1423808,...|[Glucophage, Glyc...|
 19823|2167-02-25|FREQUENCY-DRUG|FREQUENCY| 2353| 2355| bid| DRUG| 2361| 2370|Glucophage| 1.0| Glucophage| 0| Glucophage| null| 151827|[151827, 1423808,...|[Glucophage, Glyc...|
 19823|2167-02-25|FREQUENCY-DRUG|FREQUENCY| 2353| 2355| bid| DRUG| 2392| 2400| Singulair| 1.0| Singulair| 0| Singulair| null| 152894|[152894, 417282, ...|[Serdolect, neo-h...|
 19823|2167-02-25|FREQUENCY-DRUG|FREQUENCY| 2353| 2355| bid| DRUG| 2421| 2425| Zocor| 1.0| Zocor| 0| Zocor| null| 196503|[196503, 858343, ...|[Zocor, Zipsor, Z...|
 19823|2167-02-25|FREQUENCY-DRUG|FREQUENCY| 2353| 2355| bid| DRUG| 2446| 2451| Hyzaar| 1.0| Hyzaar| 0| Hyzaar| null| 217681|[217681, 1043619,...|[Hyzaar, eucalypt...|
 19823|2167-02-25|FREQUENCY-DRUG|FREQUENCY| 2353| 2355| bid| DRUG| 2471| 2478| Atenolol| 1.0| Atenolol| 0| Atenolol| null| 218896|[218896, 370619, ...|[OptiPranolol, at...|
 19823|2167-02-25| DRUG-STRENGTH| DRUG| 2361| 2370|Glucophage| STRENGTH| 2372| 2379| 1,000 mg| 1.0| Glucophage 1,000 mg| 0| Glucophage 1,000 mg| null| 153095|[153095, 1721278,...|[glucagon (rDNA) ...|
 19823|2167-02-25| DRUG-ROUTE| DRUG| 2361| 2370|Glucophage| ROUTE| 2381| 2382| po| 1.0| Glucophage| 0| Glucophage| null| 151827|[151827, 1423808,...|[Glucophage, Glyc...|
 19823|2167-02-25|DRUG-FREQUENCY| DRUG| 2361| 2370|Glucophage|FREQUENCY| 2384| 2386| bid| 1.0| Glucophage|

In [0]:
pd_rxnorm_result = sp_rxnorm_result.toPandas()
pd_rxnorm_result

Out[20]:

,subject_id,date,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence,rx_text,sent_id,ner_chunk,entity,rxnorm_code,all_codes,resolutions
0,19823,2167-02-25,DRUG-FORM,DRUG,1391,1399,Albuterol,FORM,1414,1423,nebulizers,1.0,Albuterol nebulizers,0,Albuterol nebulizers,None,2108226,"[2108226, 1154602, 370790, 1154603, 2108233, 2108255, 2108276, 745678, 1163444, 2108246, 2108507...","[albuterol Inhalation Solution, albuterol Inhalant Product, albuterol Injectable Solution, albut..."
1,19823,2167-02-25,DRUG-FORM,DRUG,1405,1412,Atrovent,FORM,1414,1423,nebulizers,1.0,Atrovent nebulizers,0,Atrovent nebulizers,None,2108451,"[2108451, 1173573, 379767, 1173576, 2463732, 1945043, 1172634, 1171309, 363357, 1184866, 1170108...","[ipratropium Inhalation Solution [Atrovent], Atrovent Inhalant Product, Atrovent Autohaler, Atro..."
2,19823,2167-02-25,STRENGTH-DRUG,STRENGTH,1539,1543,40 mg,DRUG,1551,1555,Lasix,1.0,Lasix 40 mg,0,Lasix 40 mg,None,200809,"[200809, 617319, 103919, 1871459, 201286, 2556796, 1927858, 1648194, 977916, 352320, 208458, 173...","[furosemide 40 MG Oral Tablet [Lasix], atorvastatin 40 MG [Lipitor], fluvastatin 40 MG Oral Caps..."
3,19823,2167-02-25,ROUTE-DRUG,ROUTE,1548,1549,IV,DRUG,1551,1555,Lasix,1.0,Lasix,0,Lasix,None,202991,"[202991, 151963, 2256936, 2256930, 1043720, 224946, 217961, 203783, 261550, 1013021, 606658, 218...","[Lasix, Lasma, lasmiditan Oral Tablet, lasmiditan, LidoWorx, Lidex, Laniroif, Lanoxicaps, Lanabi..."
4,19823,2167-02-25,DRUG-STRENGTH,DRUG,2336,2341,Amaryl,STRENGTH,2343,2348,2.0 mg,1.0,Amaryl 2.0 mg,0,Amaryl 2.0 mg,None,901295,"[901295, 153591, 1310138, 213799, 2399657, 1036818, 998190, 1439900, 905270, 540140, 202295, 104...","[sodium fluoride 2.2 MG [Ludent], glimepiride 2 MG Oral Tablet [Amaryl], everolimus 2 MG Tablet ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2777,70004,2182-08-05,ROUTE-DRUG,ROUTE,545,546,IV,DRUG,548,555,contrast,1.0,contrast,0,contrast,None,1592743,"[1592743, 202939, 23202, 66977, 2262255, 543455, 705766, 1436150, 744843, 216795, 1946584, 65874...","[Ofev, Dixarit, Dilor, Vascor, Scenesse, Durad, Appearex, Visco-Gel, Isentress, Duratest, Xhance..."
2778,70004,2182-08-05,DOSAGE-DRUG,DOSAGE,942,946,20 cc,DRUG,951,959,Magnevist,1.0,Magnevist 20 cc,0,Magnevist 20 cc,None,208456,"[208456, 152893, 2286257, 617317, 664142, 1119558, 596927, 429343, 440810, 571777, 351387, 79386...","[tacrine 20 MG Oral Capsule [Cognex], sertindole 20 MG Oral Tablet [Serdolect], dexamethasone 20..."
2779,70004,2182-08-05,DRUG-ROUTE,DRUG,951,959,Magnevist,ROUTE,961,971,intravenous,1.0,Magnevist,0,Magnevist,None,196214,"[196214, 2475179, 406156, 991881, 6574, 218204, 218250, 218167, 797858, 1043619, 152000, 218245,...","[Magnesiocard, magnesite, MagneBind, Maracyn Plus, magnesium, Maoson, Maxaquin, Magagel Plus, Ma..."
2780,70004,2182-08-16,ROUTE-DRUG,ROUTE,475,476,IV,DRUG,478,485,CONTRAST,1.0,CONTRAST,0,CONTRAST,None,799044,"[799044, 153381, 385716, 216281, 668395, 1013644, 216253, 284702, 1188463, 2264346, 323984, 2158...","[Cotab A, Cozaar-Comp, Cesamet, Crolom, Certuss, Cidaflex, Cosopt, Colocort, Citravet, belladonn..."


In [0]:
outname = 'posology_RE_rxnorm_results.csv'
outdir = '/dbfs/FileStore/'
dbutils.fs.ls(outdir)

Out[21]: [FileInfo(path='dbfs:/dbfs/FileStore/log/', name='log/', size=0),
 FileInfo(path='dbfs:/dbfs/FileStore/logs/', name='logs/', size=0)]

In [0]:
outname = 'posology_RE_rxnorm_results.csv'
outdir = '/dbfs/FileStore/'
dbutils.fs.rm(outdir+outname)
pd_rxnorm_result.to_csv(outdir+outname, index=False, encoding="utf-8")

### Split Resolutions to Resolution Drug

In [0]:
outname = 'posology_RE_rxnorm_results.csv'
outdir = '/dbfs/FileStore/'
df = pd.read_csv(outdir+outname)
df

Out[23]:

,subject_id,date,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence,rx_text,sent_id,ner_chunk,entity,rxnorm_code,all_codes,resolutions
0,19823,2167-02-25,DRUG-FORM,DRUG,1391,1399,Albuterol,FORM,1414,1423,nebulizers,1.0,Albuterol nebulizers,0,Albuterol nebulizers,NaN,2108226,['2108226' '1154602' '370790' '1154603' '2108233' '2108255' '2108276'\n '745678' '1163444' '2108...,['albuterol Inhalation Solution' 'albuterol Inhalant Product'\n 'albuterol Injectable Solution' ...
1,19823,2167-02-25,DRUG-FORM,DRUG,1405,1412,Atrovent,FORM,1414,1423,nebulizers,1.0,Atrovent nebulizers,0,Atrovent nebulizers,NaN,2108451,['2108451' '1173573' '379767' '1173576' '2463732' '1945043' '1172634'\n '1171309' '363357' '1184...,['ipratropium Inhalation Solution [Atrovent]' 'Atrovent Inhalant Product'\n 'Atrovent Autohaler'...
2,19823,2167-02-25,STRENGTH-DRUG,STRENGTH,1539,1543,40 mg,DRUG,1551,1555,Lasix,1.0,Lasix 40 mg,0,Lasix 40 mg,NaN,200809,['200809' '617319' '103919' '1871459' '201286' '2556796' '1927858'\n '1648194' '977916' '352320'...,['furosemide 40 MG Oral Tablet [Lasix]' 'atorvastatin 40 MG [Lipitor]'\n 'fluvastatin 40 MG Oral...
3,19823,2167-02-25,ROUTE-DRUG,ROUTE,1548,1549,IV,DRUG,1551,1555,Lasix,1.0,Lasix,0,Lasix,NaN,202991,['202991' '151963' '2256936' '2256930' '1043720' '224946' '217961'\n '203783' '261550' '1013021'...,['Lasix' 'Lasma' 'lasmiditan Oral Tablet' 'lasmiditan' 'LidoWorx' 'Lidex'\n 'Laniroif' 'Lanoxica...
4,19823,2167-02-25,DRUG-STRENGTH,DRUG,2336,2341,Amaryl,STRENGTH,2343,2348,2.0 mg,1.0,Amaryl 2.0 mg,0,Amaryl 2.0 mg,NaN,901295,['901295' '153591' '1310138' '213799' '2399657' '1036818' '998190'\n '1439900' '905270' '540140'...,['sodium fluoride 2.2 MG [Ludent]' 'glimepiride 2 MG Oral Tablet [Amaryl]'\n 'everolimus 2 MG Ta...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2777,70004,2182-08-05,ROUTE-DRUG,ROUTE,545,546,IV,DRUG,548,555,contrast,1.0,contrast,0,contrast,NaN,1592743,['1592743' '202939' '23202' '66977' '2262255' '543455' '705766' '1436150'\n '744843' '216795' '1...,['Ofev' 'Dixarit' 'Dilor' 'Vascor' 'Scenesse' 'Durad' 'Appearex'\n 'Visco-Gel' 'Isentress' 'Dura...
2778,70004,2182-08-05,DOSAGE-DRUG,DOSAGE,942,946,20 cc,DRUG,951,959,Magnevist,1.0,Magnevist 20 cc,0,Magnevist 20 cc,NaN,208456,['208456' '152893' '2286257' '617317' '664142' '1119558' '596927' '429343'\n '440810' '571777' '...,['tacrine 20 MG Oral Capsule [Cognex]'\n 'sertindole 20 MG Oral Tablet [Serdolect]' 'dexamethaso...
2779,70004,2182-08-05,DRUG-ROUTE,DRUG,951,959,Magnevist,ROUTE,961,971,intravenous,1.0,Magnevist,0,Magnevist,NaN,196214,['196214' '2475179' '406156' '991881' '6574' '218204' '218250' '218167'\n '797858' '1043619' '15...,['Magnesiocard' 'magnesite' 'MagneBind' 'Maracyn Plus' 'magnesium'\n 'Maoson' 'Maxaquin' 'Magage...
2780,70004,2182-08-16,ROUTE-DRUG,ROUTE,475,476,IV,DRUG,478,485,CONTRAST,1.0,CONTRAST,0,CONTRAST,NaN,799044,['799044' '153381' '385716' '216281' '668395' '1013644' '216253' '284702'\n '1188463' '2264346' ...,['Cotab A' 'Cozaar-Comp' 'Cesamet' 'Crolom' 'Certuss' 'Cidaflex' 'Cosopt'\n 'Colocort' 'Citravet...


In [0]:
df['res']=df['resolutions'].str.split(' ').str[0]
df.res.head()

Out[24]: 0 ['albuterol
1 ['ipratropium
2 ['furosemide
3 ['Lasix'
4 ['sodium
Name: res, dtype: object

In [0]:
df['resolution'] = [val[2:] for val in df['res']]
df['resolution'].head()

Out[25]: 0 albuterol
1 ipratropium
2 furosemide
3 Lasix'
4 sodium
Name: resolution, dtype: object

In [0]:
df['drug_resolution'] = df['resolution'].str.split().str.get(0)
df['drug_resolution'] = df['drug_resolution'].replace({',':''}, regex=True)
df['drug_resolution'] = df['drug_resolution'].replace({"'":""}, regex=True)
df.head(20)

Out[26]:

,subject_id,date,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence,rx_text,sent_id,ner_chunk,entity,rxnorm_code,all_codes,resolutions,res,resolution,drug_resolution
0,19823,2167-02-25,DRUG-FORM,DRUG,1391,1399,Albuterol,FORM,1414,1423,nebulizers,1.0,Albuterol nebulizers,0,Albuterol nebulizers,NaN,2108226,['2108226' '1154602' '370790' '1154603' '2108233' '2108255' '2108276'\n '745678' '1163444' '2108...,['albuterol Inhalation Solution' 'albuterol Inhalant Product'\n 'albuterol Injectable Solution' ...,['albuterol,albuterol,albuterol
1,19823,2167-02-25,DRUG-FORM,DRUG,1405,1412,Atrovent,FORM,1414,1423,nebulizers,1.0,Atrovent nebulizers,0,Atrovent nebulizers,NaN,2108451,['2108451' '1173573' '379767' '1173576' '2463732' '1945043' '1172634'\n '1171309' '363357' '1184...,['ipratropium Inhalation Solution [Atrovent]' 'Atrovent Inhalant Product'\n 'Atrovent Autohaler'...,['ipratropium,ipratropium,ipratropium
2,19823,2167-02-25,STRENGTH-DRUG,STRENGTH,1539,1543,40 mg,DRUG,1551,1555,Lasix,1.0,Lasix 40 mg,0,Lasix 40 mg,NaN,200809,['200809' '617319' '103919' '1871459' '201286' '2556796' '1927858'\n '1648194' '977916' '352320'...,['furosemide 40 MG Oral Tablet [Lasix]' 'atorvastatin 40 MG [Lipitor]'\n 'fluvastatin 40 MG Oral...,['furosemide,furosemide,furosemide
3,19823,2167-02-25,ROUTE-DRUG,ROUTE,1548,1549,IV,DRUG,1551,1555,Lasix,1.0,Lasix,0,Lasix,NaN,202991,['202991' '151963' '2256936' '2256930' '1043720' '224946' '217961'\n '203783' '261550' '1013021'...,['Lasix' 'Lasma' 'lasmiditan Oral Tablet' 'lasmiditan' 'LidoWorx' 'Lidex'\n 'Laniroif' 'Lanoxica...,['Lasix',Lasix',Lasix
4,19823,2167-02-25,DRUG-STRENGTH,DRUG,2336,2341,Amaryl,STRENGTH,2343,2348,2.0 mg,1.0,Amaryl 2.0 mg,0,Amaryl 2.0 mg,NaN,901295,['901295' '153591' '1310138' '213799' '2399657' '1036818' '998190'\n '1439900' '905270' '540140'...,['sodium fluoride 2.2 MG [Ludent]' 'glimepiride 2 MG Oral Tablet [Amaryl]'\n 'everolimus 2 MG Ta...,['sodium,sodium,sodium
5,19823,2167-02-25,DRUG-ROUTE,DRUG,2336,2341,Amaryl,ROUTE,2350,2351,po,1.0,Amaryl,0,Amaryl,NaN,215221,['215221' '135820' '151348' '215203' '153592' '152800' '215200' '151345'\n '131725' '215206' '83...,['Amilac' 'Aventyl' 'Amytal' 'Amcort' 'Amaryl' 'Amilamont' 'Ambenyl'\n 'Amoram' 'Ambien' 'Americ...,['Amilac',Amilac',Amilac
6,19823,2167-02-25,DRUG-FREQUENCY,DRUG,2336,2341,Amaryl,FREQUENCY,2353,2355,bid,1.0,Amaryl,0,Amaryl,NaN,215221,['215221' '135820' '151348' '215203' '153592' '152800' '215200' '151345'\n '131725' '215206' '83...,['Amilac' 'Aventyl' 'Amytal' 'Amcort' 'Amaryl' 'Amilamont' 'Ambenyl'\n 'Amoram' 'Ambien' 'Americ...,['Amilac',Amilac',Amilac
7,19823,2167-02-25,DRUG-STRENGTH,DRUG,2336,2341,Amaryl,STRENGTH,2372,2379,"1,000 mg",1.0,"Amaryl 1,000 mg",0,"Amaryl 1,000 mg",NaN,106248,['106248' '1549223' '1654725' '1298448' '282828' '885214' '1312717'\n '417424' '409160' '1293504...,['hydrocortisone 1 MG/ML Topical Cream' 'lidocaine 10 MG/ML Topical Spray'\n 'glycerin 250 MG/ML...,['hydrocortisone,hydrocortisone,hydrocortisone
8,19823,2167-02-25,DRUG-FREQUENCY,DRUG,2336,2341,Amaryl,FREQUENCY,2384,2386,bid,1.0,Amaryl,0,Amaryl,NaN,215221,['215221' '135820' '151348' '215203' '153592' '152800' '215200' '151345'\n '131725' '215206' '83...,['Amilac' 'Aventyl' 'Amytal' 'Amcort' 'Amaryl' 'Amilamont' 'Ambenyl'\n 'Amoram' 'Ambien' 'Americ...,['Amilac',Amilac',Amilac
9,19823,2167-02-25,STRENGTH-DRUG,STRENGTH,2343,2348,2.0 mg,DRUG,2361,2370,Glucophage,1.0,Glucophage 2.0 mg,0,Glucophage 2.0 mg,NaN,865570,['865570' '201058' '1855336' '2001263' '205490' '808502' '996825' '199176'\n '999493' '315321' '...,['glipizide 2.5 MG [Glucotrol]' 'glyburide 2.5 MG Oral Tablet [Euglucon]'\n 'omeprazole 2.5 MG [...,['glipizide,glipizide,glipizide


In [0]:
df['drug_resolution'] = df['drug_resolution'].str.lower()
df['chunk1'] = df['chunk1'].str.lower()
df['chunk2'] = df['chunk2'].str.lower()
df.head(20)

Out[27]:

,subject_id,date,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence,rx_text,sent_id,ner_chunk,entity,rxnorm_code,all_codes,resolutions,res,resolution,drug_resolution
0,19823,2167-02-25,DRUG-FORM,DRUG,1391,1399,albuterol,FORM,1414,1423,nebulizers,1.0,Albuterol nebulizers,0,Albuterol nebulizers,NaN,2108226,['2108226' '1154602' '370790' '1154603' '2108233' '2108255' '2108276'\n '745678' '1163444' '2108...,['albuterol Inhalation Solution' 'albuterol Inhalant Product'\n 'albuterol Injectable Solution' ...,['albuterol,albuterol,albuterol
1,19823,2167-02-25,DRUG-FORM,DRUG,1405,1412,atrovent,FORM,1414,1423,nebulizers,1.0,Atrovent nebulizers,0,Atrovent nebulizers,NaN,2108451,['2108451' '1173573' '379767' '1173576' '2463732' '1945043' '1172634'\n '1171309' '363357' '1184...,['ipratropium Inhalation Solution [Atrovent]' 'Atrovent Inhalant Product'\n 'Atrovent Autohaler'...,['ipratropium,ipratropium,ipratropium
2,19823,2167-02-25,STRENGTH-DRUG,STRENGTH,1539,1543,40 mg,DRUG,1551,1555,lasix,1.0,Lasix 40 mg,0,Lasix 40 mg,NaN,200809,['200809' '617319' '103919' '1871459' '201286' '2556796' '1927858'\n '1648194' '977916' '352320'...,['furosemide 40 MG Oral Tablet [Lasix]' 'atorvastatin 40 MG [Lipitor]'\n 'fluvastatin 40 MG Oral...,['furosemide,furosemide,furosemide
3,19823,2167-02-25,ROUTE-DRUG,ROUTE,1548,1549,iv,DRUG,1551,1555,lasix,1.0,Lasix,0,Lasix,NaN,202991,['202991' '151963' '2256936' '2256930' '1043720' '224946' '217961'\n '203783' '261550' '1013021'...,['Lasix' 'Lasma' 'lasmiditan Oral Tablet' 'lasmiditan' 'LidoWorx' 'Lidex'\n 'Laniroif' 'Lanoxica...,['Lasix',Lasix',lasix
4,19823,2167-02-25,DRUG-STRENGTH,DRUG,2336,2341,amaryl,STRENGTH,2343,2348,2.0 mg,1.0,Amaryl 2.0 mg,0,Amaryl 2.0 mg,NaN,901295,['901295' '153591' '1310138' '213799' '2399657' '1036818' '998190'\n '1439900' '905270' '540140'...,['sodium fluoride 2.2 MG [Ludent]' 'glimepiride 2 MG Oral Tablet [Amaryl]'\n 'everolimus 2 MG Ta...,['sodium,sodium,sodium
5,19823,2167-02-25,DRUG-ROUTE,DRUG,2336,2341,amaryl,ROUTE,2350,2351,po,1.0,Amaryl,0,Amaryl,NaN,215221,['215221' '135820' '151348' '215203' '153592' '152800' '215200' '151345'\n '131725' '215206' '83...,['Amilac' 'Aventyl' 'Amytal' 'Amcort' 'Amaryl' 'Amilamont' 'Ambenyl'\n 'Amoram' 'Ambien' 'Americ...,['Amilac',Amilac',amilac
6,19823,2167-02-25,DRUG-FREQUENCY,DRUG,2336,2341,amaryl,FREQUENCY,2353,2355,bid,1.0,Amaryl,0,Amaryl,NaN,215221,['215221' '135820' '151348' '215203' '153592' '152800' '215200' '151345'\n '131725' '215206' '83...,['Amilac' 'Aventyl' 'Amytal' 'Amcort' 'Amaryl' 'Amilamont' 'Ambenyl'\n 'Amoram' 'Ambien' 'Americ...,['Amilac',Amilac',amilac
7,19823,2167-02-25,DRUG-STRENGTH,DRUG,2336,2341,amaryl,STRENGTH,2372,2379,"1,000 mg",1.0,"Amaryl 1,000 mg",0,"Amaryl 1,000 mg",NaN,106248,['106248' '1549223' '1654725' '1298448' '282828' '885214' '1312717'\n '417424' '409160' '1293504...,['hydrocortisone 1 MG/ML Topical Cream' 'lidocaine 10 MG/ML Topical Spray'\n 'glycerin 250 MG/ML...,['hydrocortisone,hydrocortisone,hydrocortisone
8,19823,2167-02-25,DRUG-FREQUENCY,DRUG,2336,2341,amaryl,FREQUENCY,2384,2386,bid,1.0,Amaryl,0,Amaryl,NaN,215221,['215221' '135820' '151348' '215203' '153592' '152800' '215200' '151345'\n '131725' '215206' '83...,['Amilac' 'Aventyl' 'Amytal' 'Amcort' 'Amaryl' 'Amilamont' 'Ambenyl'\n 'Amoram' 'Ambien' 'Americ...,['Amilac',Amilac',amilac
9,19823,2167-02-25,STRENGTH-DRUG,STRENGTH,2343,2348,2.0 mg,DRUG,2361,2370,glucophage,1.0,Glucophage 2.0 mg,0,Glucophage 2.0 mg,NaN,865570,['865570' '201058' '1855336' '2001263' '205490' '808502' '996825' '199176'\n '999493' '315321' '...,['glipizide 2.5 MG [Glucotrol]' 'glyburide 2.5 MG Oral Tablet [Euglucon]'\n 'omeprazole 2.5 MG [...,['glipizide,glipizide,glipizide


In [0]:
outname = 'posology_RE_rxnorm_w_drug_resolutions.csv'
outdir = '/dbfs/FileStore/'
dbutils.fs.rm(outdir+outname)
df.to_csv(outdir+outname, index=False, encoding="utf-8")

## NER JSL Slim

In [0]:
documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("document")

sentenceDetector = SentenceDetector()\
      .setInputCols(["document"])\
      .setOutputCol("sentence")\
      .setCustomBounds(["\|"])

tokenizer = Tokenizer()\
      .setInputCols(["sentence"])\
      .setOutputCol("token")\

word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
      .setInputCols(["sentence", "token"])\
      .setOutputCol("embeddings")

jsl_ner = MedicalNerModel.pretrained("ner_jsl_slim", "en", "clinical/models") \
      .setInputCols(["sentence", "token", "embeddings"]) \
      .setOutputCol("ner")

jsl_converter = NerConverter() \
      .setInputCols(["sentence", "token", "ner"]) \
      .setOutputCol("ner_chunk")\
      .setWhiteList(['Symptom','Body_Part', 'Procedure', 'Disease_Syndrome_Disorder', 'Test'])

ner_pipeline = Pipeline(
    stages = [
        documentAssembler,
        sentenceDetector,
        tokenizer,
        word_embeddings,
        jsl_ner,
        jsl_converter
        ])

data_ner = spark.createDataFrame([[""]]).toDF("text")
model = ner_pipeline.fit(data_ner)

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
ner_jsl_slim download started this may take some time.
Approximate size to download 14.4 MB
[ | ][OK!]

In [0]:
results = model.transform(sparkDF)
results.printSchema()

root
-- subject_id: long (nullable = true)
-- date: string (nullable = true)
-- text: string (nullable = true)
-- gender: string (nullable = true)
-- dateOfBirth: string (nullable = true)
-- document: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- sentence: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- token: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- embeddings: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- ner: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- ner_chunk: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)

In [0]:
result_df = results.select('subject_id','date',
                           F.explode(F.arrays_zip(results.ner_chunk.result, results.ner_chunk.begin, results.ner_chunk.end, results.ner_chunk.metadata)).alias("cols")) \
                    .select('subject_id','date',
                            F.expr("cols['3']['sentence']").alias("sentence_id"),
                            F.expr("cols['0']").alias("chunk"),
                            F.expr("cols['1']").alias("begin"),
                            F.expr("cols['2']").alias("end"),
                            F.expr("cols['3']['entity']").alias("ner_label"))\
                    .filter("ner_label!='O'")

In [0]:
result_df.show()

+----------+----------+-----------+--------------------+-----+----+--------------------+
subject_id| date|sentence_id| chunk|begin| end| ner_label|
+----------+----------+-----------+--------------------+-----+----+--------------------+
 19823|2167-02-25| 0| Shortness of breath| 178| 196| Symptom|
 19823|2167-02-25| 0| cough| 199| 203| Symptom|
 19823|2167-02-25| 1| diabetes type II| 345| 360|Disease_Syndrome_...|
 19823|2167-02-25| 1|congestive heart ...| 363| 386|Disease_Syndrome_...|
 19823|2167-02-25| 1| hypertension| 413| 424|Disease_Syndrome_...|
 19823|2167-02-25| 2|progressively wor...| 477| 519| Symptom|
 19823|2167-02-25| 2| dyspnea on exertion| 525| 543| Symptom|
 19823|2167-02-25| 2| wheezing| 546| 553| Symptom|
 19823|2167-02-25| 2| nonproductive
cough| 556| 574| Symptom|
 19823|2167-02-25| 3| nausea| 666| 671| Symptom|
 19823|2167-02-25| 3| vomiting| 676| 683| Symptom|
 19823|2167-02-25| 3| chills| 686| 691| Symptom|
 19823|2167-02-25| 3| night sweats| 703| 714| Symptom|
 19823|2167-02-25| 3| chest pain| 720| 729| Symptom|
 19823|2167-02-25| 4| diarrhea| 744| 751| Symptom|
 19823|2167-02-25| 5| constipated| 767| 777| Symptom|
 19823|2167-02-25| 6|paroxysmal
noctur...| 803| 830| Symptom|
 19823|2167-02-25| 6| pillow orthopnea| 840| 855| Symptom|
 19823|2167-02-25| 7| dysuria| 878| 884| Symptom|
 19823|2167-02-25| 10| lethargic| 1060|1068| Symptom|
+----------+----------+-----------+--------------------+-----+----+--------------------+
only showing top 20 rows

In [0]:
pd_result = result_df.toPandas()
pd_result

Out[47]:

,subject_id,date,sentence_id,chunk,begin,end,ner_label
0,19823,2167-02-25,0,Shortness of breath,178,196,Symptom
1,19823,2167-02-25,0,cough,199,203,Symptom
2,19823,2167-02-25,1,diabetes type II,345,360,Disease_Syndrome_Disorder
3,19823,2167-02-25,1,congestive heart failure,363,386,Disease_Syndrome_Disorder
4,19823,2167-02-25,1,hypertension,413,424,Disease_Syndrome_Disorder
...,...,...,...,...,...,...,...
18456,70004,2182-08-16,13,Multilevel degenerative changes,1860,1890,Symptom
18457,70004,2182-08-16,13,uncovertebral joint hypertrophy,1897,1927,Disease_Syndrome_Disorder
18458,70004,2182-08-16,14,metastatic disease,1966,1983,Oncological
18459,70004,2182-08-16,14,cervical spine,1992,2005,Body_Part


In [0]:
outname = 'ner_jsl_slim_results.csv'
outdir = '/dbfs/FileStore/'
pd_result.to_csv(outdir+outname, index=False, encoding="utf-8")

In [0]:
temp = pd.read_csv('/dbfs/FileStore/ner_jsl_slim_results.csv')
temp

Out[49]:

,subject_id,date,sentence_id,chunk,begin,end,ner_label
0,19823,2167-02-25,0,Shortness of breath,178,196,Symptom
1,19823,2167-02-25,0,cough,199,203,Symptom
2,19823,2167-02-25,1,diabetes type II,345,360,Disease_Syndrome_Disorder
3,19823,2167-02-25,1,congestive heart failure,363,386,Disease_Syndrome_Disorder
4,19823,2167-02-25,1,hypertension,413,424,Disease_Syndrome_Disorder
...,...,...,...,...,...,...,...
18456,70004,2182-08-16,13,Multilevel degenerative changes,1860,1890,Symptom
18457,70004,2182-08-16,13,uncovertebral joint hypertrophy,1897,1927,Disease_Syndrome_Disorder
18458,70004,2182-08-16,14,metastatic disease,1966,1983,Oncological
18459,70004,2182-08-16,14,cervical spine,1992,2005,Body_Part


# Creation of the Knowledge Graph

## Neo4j Connection

In [0]:
from neo4j import GraphDatabase
import time
from tqdm import tqdm
import pandas as pd
import json

In [0]:
patient_df = pd.read_csv('/dbfs/data.csv', sep=';')
patient_df

Out[3]:

,subject_id,date,text,gender,dateOfBirth
0,19823,2167-02-25,Admission Date: [**2167-2-16**] Dischar...,F,2099-05-05
1,19823,2167-11-27,Admission Date: [**2167-11-27**] Discha...,F,2099-05-05
2,19823,2170-10-12,Admission Date: [**2170-9-19**] ...,F,2099-05-05
3,19823,2172-06-22,Admission Date: [**2172-6-13**] ...,F,2099-05-05
4,19823,2167-12-07,PATIENT/TEST INFORMATION:\nIndication: Aortic ...,F,2099-05-05
...,...,...,...,...,...
960,70004,2182-06-14,[**2182-6-14**] 10:45 AM\n MR HEAD W & W/O CON...,M,2127-12-06
961,70004,2182-06-25,FDG TUMOR IMAGING (PET-CT) ...,M,2127-12-06
962,70004,2182-08-05,[**2182-8-5**] 11:46 AM\n MR HEAD W & W/O CONT...,M,2127-12-06
963,70004,2182-08-23,FDG TUMOR IMAGING (PET-CT) ...,M,2127-12-06


In [0]:
patient_demographics = patient_df[['subject_id', 'gender', 'dateOfBirth']].drop_duplicates().reset_index(drop=True)
patient_demographics

Out[4]:

,subject_id,gender,dateOfBirth
0,19823,F,2099-05-05
1,22015,M,2085-10-04
2,17494,F,2193-10-01
3,21153,M,2057-03-03
4,12200,M,2136-06-17
5,23266,M,2122-03-16
6,75632,M,2118-09-04
7,27386,M,2114-08-20
8,28552,F,2044-10-22
9,70004,M,2127-12-06


In [0]:
dir = '/dbfs/FileStore/'
file = 'posology_RE_rxnorm_w_drug_resolutions.csv'
pos_RE_result =  pd.read_csv(dir+file)
pos_RE_result.head()

Out[5]:

,subject_id,date,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence,rx_text,sent_id,ner_chunk,entity,rxnorm_code,all_codes,resolutions,res,resolution,drug_resolution
0,19823,2167-02-25,DRUG-FORM,DRUG,1391,1399,albuterol,FORM,1414,1423,nebulizers,1.0,Albuterol nebulizers,0,Albuterol nebulizers,NaN,2108226,['2108226' '1154602' '370790' '1154603' '21082...,['albuterol Inhalation Solution' 'albuterol In...,['albuterol,albuterol,albuterol
1,19823,2167-02-25,DRUG-FORM,DRUG,1405,1412,atrovent,FORM,1414,1423,nebulizers,1.0,Atrovent nebulizers,0,Atrovent nebulizers,NaN,2108451,['2108451' '1173573' '379767' '1173576' '24637...,['ipratropium Inhalation Solution [Atrovent]' ...,['ipratropium,ipratropium,ipratropium
2,19823,2167-02-25,STRENGTH-DRUG,STRENGTH,1539,1543,40 mg,DRUG,1551,1555,lasix,1.0,Lasix 40 mg,0,Lasix 40 mg,NaN,200809,['200809' '617319' '103919' '1871459' '201286'...,['furosemide 40 MG Oral Tablet [Lasix]' 'atorv...,['furosemide,furosemide,furosemide
3,19823,2167-02-25,ROUTE-DRUG,ROUTE,1548,1549,iv,DRUG,1551,1555,lasix,1.0,Lasix,0,Lasix,NaN,202991,['202991' '151963' '2256936' '2256930' '104372...,['Lasix' 'Lasma' 'lasmiditan Oral Tablet' 'las...,['Lasix',Lasix',lasix
4,19823,2167-02-25,DRUG-STRENGTH,DRUG,2336,2341,amaryl,STRENGTH,2343,2348,2.0 mg,1.0,Amaryl 2.0 mg,0,Amaryl 2.0 mg,NaN,901295,['901295' '153591' '1310138' '213799' '2399657...,['sodium fluoride 2.2 MG [Ludent]' 'glimepirid...,['sodium,sodium,sodium


In [0]:
file = 'ner_jsl_slim_results.csv'
ner_DF_result = pd.read_csv(dir+file)
ner_DF_result.head()

Out[12]:

,subject_id,date,sentence_id,chunk,begin,end,ner_label
0,19823,2167-02-25,0,Shortness of breath,178,196,Symptom
1,19823,2167-02-25,0,cough,199,203,Symptom
2,19823,2167-02-25,1,diabetes type II,345,360,Disease_Syndrome_Disorder
3,19823,2167-02-25,1,congestive heart failure,363,386,Disease_Syndrome_Disorder
4,19823,2167-02-25,1,hypertension,413,424,Disease_Syndrome_Disorder


In [0]:
class Neo4jConnection:
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [0]:
uri = 'bolt://44.195.22.173:7687' 
pwd = 'hatchets-oxygen-tachometer'
user = 'neo4j'

conn = Neo4jConnection(uri=uri, user=user , pwd=pwd)

In [0]:
conn.query('CREATE CONSTRAINT patients IF NOT EXISTS ON (p:Patient) ASSERT p.name IS UNIQUE;')
conn.query('CREATE CONSTRAINT rx_norm_codes IF NOT EXISTS ON (rx:RxNorm) ASSERT rx.code IS UNIQUE;')
conn.query('CREATE CONSTRAINT drugs IF NOT EXISTS ON (drug:Drug) ASSERT drug.name IS UNIQUE;')
conn.query('CREATE CONSTRAINT ners IF NOT EXISTS ON (n:NER) ASSERT n.name IS UNIQUE;')
conn.query('CREATE CONSTRAINT symptoms IF NOT EXISTS ON (s:Symptom) ASSERT s.name IS UNIQUE;')
conn.query('CREATE CONSTRAINT bodyParts IF NOT EXISTS ON (bp:BodyPart) ASSERT bp.name IS UNIQUE;')
conn.query('CREATE CONSTRAINT procedures IF NOT EXISTS ON (p:Procedure) ASSERT p.name IS UNIQUE;')
conn.query('CREATE CONSTRAINT tests IF NOT EXISTS ON (t:Test) ASSERT t.name IS UNIQUE;')
conn.query('CREATE CONSTRAINT dsds IF NOT EXISTS ON (dsd:DSD) ASSERT dsd.name IS UNIQUE;')

Out[7]: []

In [0]:
def update_data(query, rows, batch_size = 10000):
    total = 0
    batch = 0
    start = time.time()
    result = None
    while batch * batch_size < len(rows):
        res = conn.query(query, parameters={'rows': rows[batch*batch_size:(batch+1)*batch_size].to_dict('records')})
        total += res[0]['total']
        batch += 1
        result = {"total":total, "batches":batch, "time":time.time()-start}
        print(result)
    return result

In [0]:
def add_patients(rows, batch_size=10000):
    query = '''
    UNWIND $rows as row
    MERGE(p:Patient{name:row.subject_id}) 
    ON CREATE SET p.gender      = row.gender,
                  p.dateOfBirth = row.dateOfBirth

    WITH p
    MATCH (p)
    RETURN count(*) as total
    '''
    return update_data(query, rows, batch_size)

add_patients(patient_demographics)

{'total': 10, 'batches': 1, 'time': 0.11297821998596191}
Out[9]: {'total': 10, 'batches': 1, 'time': 0.11297821998596191}

In [0]:
def add_drugs_ners(rows, batch_size=1000):
    query = '''
    UNWIND $rows as row
    
    MERGE(p:Patient{name:row.subject_id}) 
    MERGE(rx:RxNorm{code:row.rxnorm_code})
    MERGE (p)-[:RXNORM_CODE{date:date(row.date)}]->(rx)
    
    MERGE (d:Drug{name:row.drug_resolution})
    MERGE (rx)-[:DRUG_GENERIC{date:date(row.date), patient_name:row.subject_id}]->(d)
    
    MERGE(n1:NER{name:row.chunk1}) ON CREATE SET n1.type=row.entity1
    MERGE(n2:NER{name:row.chunk2}) ON CREATE SET n2.type=row.entity2
    
    WITH *
    MATCH (d:Drug{name:row.drug_resolution}), (n1:NER{name:row.chunk1}), (n2:NER{name:row.chunk2})
    CALL apoc.create.relationship (d,row.entity1, {patient_name:row.subject_id, date:date(row.date)}, n1) YIELD rel as relx
    CALL apoc.create.relationship (d,row.entity2, {patient_name:row.subject_id, date:date(row.date)}, n2) YIELD rel as rely
    
    WITH d
    MATCH (d)
    RETURN count(*) as total  
    '''
    return update_data(query, rows, batch_size)
  
add_drugs_ners(pos_RE_result)

{'total': 1000, 'batches': 1, 'time': 4.331569671630859}
{'total': 2000, 'batches': 2, 'time': 9.040156364440918}
{'total': 2782, 'batches': 3, 'time': 11.14558482170105}
Out[10]: {'total': 2782, 'batches': 3, 'time': 11.14558482170105}

In [0]:
# spliting dataframe into multiple dataframe iaw ner_label
grouped        = ner_DF_result.groupby('ner_label')

df_symptom     = grouped.get_group('Symptom')
df_dsd         = grouped.get_group('Disease_Syndrome_Disorder')
df_test        = grouped.get_group('Test')
df_bodyPart    = grouped.get_group('Body_Part')
df_procedure   = grouped.get_group('Procedure')

In [0]:
def add_symptoms(rows, batch_size=500):
    query = '''
    UNWIND $rows as row
    MATCH(p:Patient{name:row.subject_id})
    MERGE(n:Symptom {name:row.chunk})
    MERGE (p)-[:IS_SYMPTOM{date:date(row.date)}]->(n)

    WITH n
    MATCH (n)
    RETURN count(*) as total  
    '''
    return update_data(query, rows, batch_size)
  
add_symptoms(df_symptom)

{'total': 500, 'batches': 1, 'time': 0.4843027591705322}
{'total': 1000, 'batches': 2, 'time': 0.8330910205841064}
{'total': 1500, 'batches': 3, 'time': 1.1354827880859375}
{'total': 2000, 'batches': 4, 'time': 1.3480288982391357}
{'total': 2500, 'batches': 5, 'time': 1.5601437091827393}
{'total': 3000, 'batches': 6, 'time': 1.862185001373291}
{'total': 3500, 'batches': 7, 'time': 2.0643417835235596}
{'total': 4000, 'batches': 8, 'time': 2.2615597248077393}
{'total': 4500, 'batches': 9, 'time': 2.532264471054077}
{'total': 5000, 'batches': 10, 'time': 2.7331066131591797}
{'total': 5036, 'batches': 11, 'time': 2.8320295810699463}
Out[14]: {'total': 5036, 'batches': 11, 'time': 2.8320295810699463}

In [0]:
def add_dsds(rows, batch_size=500):
    query = '''
    UNWIND $rows as row
    MATCH(p:Patient{name:row.subject_id})
    MERGE(n:DSD {name:row.chunk})
    MERGE (p)-[:IS_DSD{date:date(row.date)}]->(n)

    WITH n
    MATCH (n)
    RETURN count(*) as total  
    '''
    return update_data(query, rows, batch_size)
  
add_dsds(df_dsd)

{'total': 500, 'batches': 1, 'time': 0.5363547801971436}
{'total': 1000, 'batches': 2, 'time': 0.7881300449371338}
{'total': 1500, 'batches': 3, 'time': 0.971163272857666}
{'total': 2000, 'batches': 4, 'time': 1.10390305519104}
{'total': 2500, 'batches': 5, 'time': 1.4679605960845947}
{'total': 2798, 'batches': 6, 'time': 1.7660226821899414}
Out[15]: {'total': 2798, 'batches': 6, 'time': 1.7660226821899414}

In [0]:
def add_tests(rows, batch_size=500):
    query = '''
    UNWIND $rows as row
    MATCH(p:Patient{name:row.subject_id})
    MERGE(n:Test {name:row.chunk})
    MERGE (p)-[:IS_TEST{date:date(row.date)}]->(n)

    WITH n
    MATCH (n)
    RETURN count(*) as total  
    '''
    return update_data(query, rows, batch_size)
  
add_tests(df_test)

{'total': 500, 'batches': 1, 'time': 0.39699244499206543}
{'total': 1000, 'batches': 2, 'time': 0.6146259307861328}
{'total': 1500, 'batches': 3, 'time': 1.009399175643921}
{'total': 2000, 'batches': 4, 'time': 1.292588233947754}
{'total': 2500, 'batches': 5, 'time': 1.4985682964324951}
{'total': 3000, 'batches': 6, 'time': 1.6966218948364258}
{'total': 3196, 'batches': 7, 'time': 1.8073601722717285}
Out[16]: {'total': 3196, 'batches': 7, 'time': 1.8073601722717285}

In [0]:
def add_bodyParts(rows, batch_size=500):
    query = '''
    UNWIND $rows as row
    MATCH(p:Patient{name:row.subject_id})
    MERGE(n:BodyPart {name:row.chunk})
    MERGE (p)-[:IS_BODYPART{date:date(row.date)}]->(n)

    WITH n
    MATCH (n)
    RETURN count(*) as total
    '''
    return update_data(query, rows, batch_size)
  
add_bodyParts(df_bodyPart)

{'total': 500, 'batches': 1, 'time': 0.45174646377563477}
{'total': 1000, 'batches': 2, 'time': 0.6665709018707275}
{'total': 1500, 'batches': 3, 'time': 0.928584098815918}
{'total': 2000, 'batches': 4, 'time': 1.1381957530975342}
{'total': 2500, 'batches': 5, 'time': 1.263178825378418}
{'total': 3000, 'batches': 6, 'time': 1.4509198665618896}
{'total': 3500, 'batches': 7, 'time': 1.5821709632873535}
{'total': 4000, 'batches': 8, 'time': 1.7095832824707031}
{'total': 4106, 'batches': 9, 'time': 1.7992794513702393}
Out[17]: {'total': 4106, 'batches': 9, 'time': 1.7992794513702393}

In [0]:
def add_procedures(rows, batch_size=500):
    query = '''
    UNWIND $rows as row
    MATCH(p:Patient{name:row.subject_id})
    MERGE(n:Procedure {name:row.chunk})
    MERGE (p)-[:IS_PROCEDURE{date:date(row.date)}]->(n)

    WITH n
    MATCH (n)
    RETURN count(*) as total  
    '''
    return update_data(query, rows, batch_size)
  
add_procedures(df_procedure)

{'total': 500, 'batches': 1, 'time': 0.25766968727111816}
{'total': 1000, 'batches': 2, 'time': 0.41423821449279785}
{'total': 1385, 'batches': 3, 'time': 0.530022144317627}
Out[18]: {'total': 1385, 'batches': 3, 'time': 0.530022144317627}

In [0]:
query_string = '''
CALL db.labels() YIELD label
CALL apoc.cypher.run('MATCH (:`'+label+'`) RETURN count(*) as count',{}) YIELD value
RETURN label, value.count as size
'''
df_nodes = pd.DataFrame([dict(_) for _ in conn.query(query_string)])
df_nodes

Out[19]:

,label,size
0,Patient,10
1,RxNorm,1068
2,Drug,680
3,NER,897
4,Symptom,2609
5,BodyPart,1205
6,Procedure,642
7,Test,1248
8,DSD,1117


In [0]:
query_string = '''
CALL db.relationshipTypes() YIELD relationshipType as type
CALL apoc.cypher.run('MATCH ()-[:`'+type+'`]->() RETURN count(*) as count',{}) YIELD value
RETURN type, value.count as size
'''
df_relationships = pd.DataFrame([dict(_) for _ in conn.query(query_string)])
df_relationships

Out[20]:

,type,size
0,RXNORM_CODE,1665
1,DRUG_GENERIC,1665
2,DRUG,2782
3,FORM,241
4,STRENGTH,791
5,ROUTE,930
6,FREQUENCY,487
7,DOSAGE,320
8,DURATION,13
9,IS_SYMPTOM,4080


<img src="https://raw.githubusercontent.com/iamvarol/blogposts/main/databricks/images/db_viz.png">

# Queries

In [0]:
# patient prescriptions
patient_name = '21153'
query_part1 = 'MATCH (p:Patient)-[rel_rx]->(rx:RxNorm)-[rel_d]->(d:Drug)-[rel_n]->(n:NER) ' #  
query_part2 = f'WHERE p.name ={patient_name} AND rel_n.date=rel_rx.date AND rel_n.patient_name=p.name ' # 
query_part3 = '''RETURN DISTINCT
                 p.name as patient_name,
                 rel_rx.date as date,
                 d.name as drug_generic_name,  
                 rx.code as rxnorm_code,
                 COALESCE(n.name,'') +  "(" + COALESCE (type(rel_n), "") + ")" as details
                 '''
query_string = query_part1 + query_part2 + query_part3

df = pd.DataFrame([dict(_) for _ in conn.query(query_string)])
df = df.drop_duplicates(subset= ['patient_name', 'date', 'drug_generic_name'])
df = df.groupby(['patient_name', 'date', 'drug_generic_name', 'rxnorm_code']).agg(lambda x: ' '.join(x)).reset_index()
df

Out[21]:

,patient_name,date,drug_generic_name,rxnorm_code,details
0,21153,2109-12-17,dex4,607868,mso4(DRUG)
1,21153,2109-12-17,everolimus,1310138,2 units(DOSAGE)
2,21153,2109-12-18,cidaflex,1013644,po(ROUTE)
3,21153,2109-12-18,clorsulon,1006411,po(ROUTE)
4,21153,2109-12-18,crolom,216281,hcl(DRUG)
...,...,...,...,...,...
130,21153,2110-10-20,cotab,799044,contrast(DRUG)
131,21153,2110-10-20,dibucaine,335534,1%(STRENGTH)
132,21153,2110-10-20,ofev,1592743,contrast(DRUG)
133,21153,2110-12-02,10,1807637,10 cc(DOSAGE)


In [0]:
# a patient's journey
patient_name = '21153'

query_part1 = f'MATCH (p:Patient)-[r1:IS_SYMPTOM]->(s:Symptom) WHERE p.name = {patient_name} '
query_part2 = '''
WITH DISTINCT p.name as patients, r1.date as dates, COLLECT(DISTINCT s.name) as symptoms, COUNT(DISTINCT s.name) as num_symptoms

MATCH (p:Patient)-[r2:IS_PROCEDURE]->(pr:Procedure)
WHERE p.name=patients AND r2.date = dates

WITH DISTINCT p.name as patients, r2.date as dates, COLLECT(DISTINCT pr.name) as procedures, COUNT(DISTINCT pr.name) as num_procedures, symptoms, num_symptoms
MATCH (p:Patient)-[r3:IS_DSD]->(_d:DSD) 
WHERE p.name=patients AND r3.date = dates

WITH DISTINCT p.name as patients, r3.date as dates, symptoms, num_symptoms, procedures, num_procedures,  COLLECT(DISTINCT _d.name) as dsds, COUNT(DISTINCT _d.name) as num_dsds
MATCH (p:Patient)-[r4:IS_TEST]->(_t:Test) 
WHERE p.name=patients AND r4.date = dates

WITH DISTINCT p.name as patients, r4.date as dates, symptoms, num_symptoms, procedures, num_procedures, dsds, num_dsds, COLLECT(_t.name) as tests, COUNT(_t.name) as num_tests
MATCH (p:Patient)-[r5:RXNORM_CODE]->(rx:RxNorm)-[r6]->(_d:Drug)
WHERE p.name=patients AND r5.date = dates
RETURN DISTINCT p.name as patients, r5.date as dates, symptoms, num_symptoms, procedures, num_procedures, dsds, num_dsds, tests, num_tests, COLLECT(DISTINCT toLower(_d.name)) as drugs, COUNT(DISTINCT toLower(_d.name)) as num_drugs, COLLECT(DISTINCT rx.code) as rxnorms, COUNT(DISTINCT rx.code) as num_rxnorm
ORDER BY dates;
'''
query_string = query_part1 + query_part2
df = pd.DataFrame([dict(_) for _ in conn.query(query_string)])
df

Out[22]:

,patients,dates,symptoms,num_symptoms,procedures,num_procedures,dsds,num_dsds,tests,num_tests,drugs,num_drugs,rxnorms,num_rxnorm
0,21153,2109-12-17,"[enviromental - rash, unresponsive, flatus, pa...",18,"[thyraloglossal cyst removal, serosagiuos drai...",9,"[esophgeal varices, cirrhosis, METABOLIC ALKAL...",7,"[MAINTAIN FSBG, SvO2, PA, POST PLT CT, INR REM...",18,"[everolimus, dex4]",2,"[1310138, 607868]",2
1,21153,2109-12-18,"[pain, Weak gag, sob, Pain, slight abd pain, s...",19,"[Suction, removal of swan ganz catheter, Liver...",4,"[ERYTHEMA, DSD, FORGETFUL AT TIMES, PH DOWN, D...",10,"[ABG, glucose, magnesium, ABG'S CLOSELY, Repla...",12,"[dex4, insulin, clorsulon, cidaflex, crolom]",5,"[607868, 1740938, 1006411, 1013644, 216281]",5
2,21153,2109-12-19,"[aspiration voice clear after swallowing, slig...",25,"[Anticipate extubation, Tolerating extubation,...",4,"[metabolic alkalosis, Assit with pulmonary tio...",6,"[IVP, 7p-7a, ABG - metabolic alkolotic, LS CTA...",9,"[tobi, dimethicone, clonidine, diamox, insulin...",6,"[220329, 1310821, 1360120, 151619, 139825, 211...",6
3,21153,2109-12-20,"[C/O SOB, SOB, N/V]",3,"[INCISION - STAPLES INTACT, LUNGS CTA BILAT]",2,"[MIN C/O PAIN, DIMINISHED AT BASES, ABG'S ACCE...",3,"[LS CTA, INCLUDING PLT CT, MIN ASSIST, HCL GTT...",6,[x-seb],1,[220956],1
4,21153,2109-12-21,"[flatus, Flat affect, nonproductive cough, bm,...",17,[bile drainage],1,[Hypertensive],1,"[cyclosporin levels, HRR, Plt, Phos, Hct, RA, ...",9,[insulin],1,[378841],1
5,21153,2109-12-28,"[estimated\nblood loss, acute\ndistress, pain,...",15,"[tonsillectomy, extubation, liver transplant, ...",7,"[numerous spider nevi, chronic hepatitis\nC, t...",9,"[T tube study, platelet count, liver function ...",32,"[percocet, p2e1, citalopram, pantoprazole, cal...",36,"[42844, 219017, 329444, 330396, 369633, 2599, ...",46
6,21153,2110-01-03,"[opacification, ductal dilatation]",2,[liver transplant],1,"[LIVER TX,ABD PAIN, stricture]",2,"[LFTs, fluoroscopic\n guidance]",2,"[slow, dibucaine]",2,"[154995, 3339]",2
7,21153,2110-01-14,"[obstruction, mild intrahepatic biliary dilata...",10,"[liver transplant, transplant liver]",2,"[abcess, Hematoma, biloma, Mid common bile\n d...",6,"[pulsed Doppler images, eval liver perfusion, ...",6,[lisinopril],1,[201382],1
8,21153,2110-01-15,[drainage],1,"[drainage, placement of 8.5 Fr percutaneous tr...",9,"[stricture, LIVER TRANSPLANT, ampullary strict...",10,"[Tube cholangiogram, Cholangiogram, cholangiog...",3,[lidocaine],1,[439728],1
9,21153,2110-01-19,"[several prominent lymph nodes, Large subscapu...",14,"[percutaneous tube placement, PTC drain placem...",2,"[ascites, LIVER TRANSPLANT, PTC PLACEMENT, bil...",4,"[Contrast-enhanced CT, CT 100CC, CT OF THE PEL...",5,"[acalabrutinib, docusate, cotab, ofev]",4,"[1986815, 1302275, 799044, 1592743]",4


<img src="https://raw.githubusercontent.com/iamvarol/blogposts/main/databricks/images/patients_journey.png">

In [0]:
# drug based query
drug_generic_name = 'isosorbide' 

query_part1 = 'MATCH (p:Patient)-[rel_rx]->(rx:RxNorm)-[rel_d]->(d:Drug)-[rel_n]->(n:NER) '
query_part2 = f'WHERE d.name ="{drug_generic_name}" AND rel_n.date=rel_rx.date AND rel_n.patient_name=p.name '
query_part3 = '''RETURN DISTINCT
                 d.name as drug_generic_name, 
                 p.name as patient_name, 
                 rel_rx.date as date, 
                 rx.code as rxnorm_code, 
                 COALESCE(n.name,'') +  "(" + COALESCE (type(rel_n), "") + ")" as details'''

query_string = query_part1 + query_part2 + query_part3
df = pd.DataFrame([dict(_) for _ in conn.query(query_string)])
df = df.groupby(['patient_name', 'date', 'rxnorm_code','drug_generic_name']).agg(lambda x : ' '.join(x)).reset_index()
df

Out[23]:

,patient_name,date,rxnorm_code,drug_generic_name,details
0,12200,2136-06-20,153689,isosorbide,piv(DRUG) 30cc/kg(STRENGTH)
1,19823,2167-12-01,201438,isosorbide,60meq(DRUG) x1(FREQUENCY) po(ROUTE)
2,19823,2172-06-20,201415,isosorbide,medicated(DRUG) 5 mg(STRENGTH)
3,21153,2110-09-09,153689,isosorbide,contrast(DRUG) 30 cc(DOSAGE)
4,22015,2161-05-05,153689,isosorbide,30 meq(STRENGTH) kphos(DRUG)


In [0]:
# patients who is prescribed lasix between May 2060 and May 2125
query_string ='''
MATCH (p:Patient)-[rel_rx]->(rx:RxNorm)-[rel_d]->(d:Drug)-[rel_n:DRUG]->(n:NER)
WHERE d.name IN ['lasix']
      AND rel_n.patient_name=p.name
      AND rel_n.date=rel_rx.date 
      AND rel_rx.date >= date("2060-05-01")
      AND rel_n.date >= date("2060-05-01")
      AND rel_rx.date < date("2125-05-01")
      AND rel_n.date < date("2125-05-01")
RETURN DISTINCT
      d.name as drug_generic_name, 
      p.name as patient_name, 
      rel_rx.date as date
ORDER BY date ASC
'''

df = pd.DataFrame([dict(_) for _ in conn.query(query_string)])
df

Out[24]:

,drug_generic_name,patient_name,date
0,lasix,28552,2122-07-17
1,lasix,28552,2122-07-18
2,lasix,28552,2122-07-20
3,lasix,28552,2122-07-21
4,lasix,28552,2122-07-22
5,lasix,28552,2122-07-24
6,lasix,28552,2122-07-29


<img src="https://raw.githubusercontent.com/iamvarol/blogposts/main/databricks/images/lasix.png">

In [0]:
# patients using warfarin 2mg and up

query_string ='''
MATCH (p:Patient)-[rel_rx]->(rx:RxNorm)-[rel_d]->(d:Drug)-[rel_n:STRENGTH]->(n:NER)
WHERE toLower(d.name)='warfarin'
      AND rel_n.patient_name=p.name
      AND rel_n.date=rel_rx.date 
      AND toInteger(left(n.name,1)) >=2
RETURN  DISTINCT
      d.name as drug_generic_name,
      rx.code as rxnorm_code,
      p.name as patient_name,
      n.name as strength,
      rel_rx.date as date
'''
df = pd.DataFrame([dict(_) for _ in conn.query(query_string)])
df

Out[25]:

,drug_generic_name,rxnorm_code,patient_name,strength,date
0,warfarin,855313,19823,2.5 mg,2167-11-27


In [0]:
# dangerous drug combinations

query_string ='''
WITH ["ibuprofen", "naproxen", "diclofenac", "indometacin", "ketorolac", "aspirin", "ketoprofen", "dexketoprofen", "meloxicam"] AS nsaids
MATCH (p:Patient)-[r1:RXNORM_CODE]->(rx:RxNorm)-[r2]->(d:Drug)
WHERE any(word IN nsaids WHERE d.name CONTAINS word) 
WITH DISTINCT p.name as patients, COLLECT(DISTINCT d.name) as nsaid_drugs, COUNT(DISTINCT d.name) as num_nsaids
MATCH (p:Patient)-[r1:RXNORM_CODE]->(rx:RxNorm)-[r2]->(d:Drug)
WHERE p.name=patients AND d.name='warfarin'
RETURN DISTINCT patients, 
                nsaid_drugs, 
                num_nsaids, 
                d.name as warfarin_drug, 
                r1.date as date
'''

df = pd.DataFrame([dict(_) for _ in conn.query(query_string)])
df

Out[26]:

,patients,nsaid_drugs,num_nsaids,warfarin_drug,date
0,19823,"[diclofenac, aspirin, ketoprofen]",3,warfarin,2172-06-22
1,19823,"[diclofenac, aspirin, ketoprofen]",3,warfarin,2167-11-27
2,19823,"[diclofenac, aspirin, ketoprofen]",3,warfarin,2170-10-12


<img src="https://raw.githubusercontent.com/iamvarol/blogposts/main/databricks/images/ddc.png">

In [0]:
# patients who underwent a hernia repair or appendectomy or cholecystectomy
query_string = """
MATCH (pcd1:Procedure)-[rel1:IS_PROCEDURE]-(pati1:Patient)
WHERE pcd1.name CONTAINS 'hernia repair' OR pcd1.name CONTAINS 'appendectomy' OR pcd1.name CONTAINS 'cholecystectomy'
RETURN DISTINCT pati1.name as patients, 
                COLLECT(DISTINCT toLower(pcd1.name)) as procedures
"""

df = pd.DataFrame([dict(_) for _ in conn.query(query_string)])
df

Out[27]:

,patients,procedures
0,21153,"[hernia repair, appendectomy]"


In [0]:
# patients with chest pain and shortness of breath
query_string = """
MATCH (p1:Patient)-[r1:IS_SYMPTOM]->(s1:Symptom),
(p2:Patient)-[r2:IS_SYMPTOM]->(s2:Symptom)
WHERE s1.name CONTAINS "chest pain" AND s2.name CONTAINS "shortness of breath"
    AND p2.name=p1.name AND r2.date = r1.date
RETURN DISTINCT p1.name as patient, r1.date as date,s1.name as symptom1, s2.name as symptom2
ORDER BY patient
"""
df = pd.DataFrame([dict(_) for _ in conn.query(query_string)])
df

Out[28]:

,patient,date,symptom1,symptom2
0,19823,2170-10-12,chest pain,shortness of breath
1,19823,2167-02-25,chest pain,shortness of breath


In [0]:
# Patients with hypertension or diabetes with chest pain

query_string = """
MATCH (p:Patient)-[r:IS_SYMPTOM]->(s:Symptom),
(p1:Patient)-[r2:IS_DSD]->(_dsd:DSD)
WHERE s.name CONTAINS "chest pain" AND p1.name=p.name AND _dsd.name IN ['hypertension', 'diabetes'] AND r2.date=r.date
RETURN DISTINCT p.name as patient, r.date as date, _dsd.name as dsd, s.name as symptom
"""
df = pd.DataFrame([dict(_) for _ in conn.query(query_string)])
df

Out[29]:

,patient,date,dsd,symptom
0,27386,2189-07-06,hypertension,chest pain
1,19823,2167-02-25,hypertension,chest pain


<img src="https://raw.githubusercontent.com/iamvarol/blogposts/main/databricks/images/chest_pain.png">